# Variance contribution figures

In this notebook, I attribute the variance seen in prices to different parameters. The gameplan is to perform regression on prices against parameter at each point in time, and plot the ratio of the variance attributable (i.e., the $r^2$ value of each regression) in time. The parameter with the highest $r^2$ value at each time is said to "drive" the prices at that time.

This notebook is a little sloppy, but I tried to clean it up the best I could.

## Import necessary packages and set plotting params

In [ ]:
import sys
import datetime
import os

import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
import seaborn as sns

# plotting parameters
color_list = ['#000000', '#E69F00', '#56B4E9', '#009E73', '#F0E442', '#0072B2', '#D55E00', '#CC79A7'] * 2
marker_list = ['o', 's', 'P', '*', '<', 'v', '>', '^', 'p']
markersize = 6
linestyle_list = ['solid', 'solid', 'solid', 'solid'] * 4
linewidth = 2
node_times = [2020, 2030, 2060, 2100, 2150, 2200]
x_label = 'Year'

%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
cdds_params={'axes.linewidth': 3,
 'axes.axisbelow': False,
 'axes.edgecolor': 'black',
 'axes.facecolor': 'None',
 'axes.grid': False,
 'axes.labelcolor': 'black',
 'axes.spines.right': False,
 'axes.spines.top': False,
 'axes.titlesize': 20,
 'axes.labelsize': 20,
 'axes.titlelocation': 'left',
 'figure.facecolor': 'white',
 'figure.figsize': (18, 10),
 'lines.solid_capstyle': 'round',
 'lines.linewidth': 2.5,
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'text.color': 'black',
 'legend.frameon': False,
 'xtick.bottom': True,
 'xtick.major.width': 3,
 'xtick.major.size': 6,
 'xtick.color': 'black',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'black',
 'ytick.direction': 'out',
 'ytick.left': True,
 'ytick.right': False,
 'ytick.color' : 'black',
 'ytick.major.width': 3, 
 'ytick.major.size': 6,
 'axes.prop_cycle': plt.cycler(linestyle=linestyle_list),
 'font.size': 16,
 'font.family': 'serif'}
plt.rcParams.update(cdds_params)

# make base filename
today = datetime.datetime.now()
year = str(today.year)
day = str(today.day)
month = str(today.month)

basefile = ''.join([month, '-', day, '-', year, '-'])

ssp5_cind = 0
ssp4_cind = 6
ssp3_cind = 7
ssp2_cind = 2
ssp1_cind = 3
alpha = 0.2

# Save figs?!

In [ ]:
save_fig = False

## Import data

Also, remove `nan` values and runs with too high utility (these tend to be unphysical).

In [ ]:
def remove_nans(ssp_paths_and_util, exp_m, temp, co2, dam, params):
    c2020, c2030, c2060, c2100, c2150, c2200, util = ssp_paths_and_util.T
    paths = ssp_paths_and_util[:, :-1]
    unphys_util_vals = util > 15
    c2020 = c2020[~unphys_util_vals]
    c2030 = c2030[~unphys_util_vals]
    c2060 = c2060[~unphys_util_vals]
    c2100 = c2100[~unphys_util_vals]
    c2150 = c2150[~unphys_util_vals]
    c2200 = c2200[~unphys_util_vals]
    util = util[~unphys_util_vals]
    paths = paths[~unphys_util_vals]
    params = params[~unphys_util_vals]
    exp_m = exp_m[~unphys_util_vals]
    temp = temp[~unphys_util_vals]
    co2 = co2[~unphys_util_vals]
    dam = dam[~unphys_util_vals]
    return paths[~np.isnan(util)], c2020[~np.isnan(util)], c2030[~np.isnan(util)], c2060[~np.isnan(util)], c2100[~np.isnan(util)], c2150[~np.isnan(util)], c2200[~np.isnan(util)], util[~np.isnan(util)], params[~np.isnan(util)], exp_m[~np.isnan(util)], temp[~np.isnan(util)], co2[~np.isnan(util)], dam[~np.isnan(util)]

path = os.getcwd()

# sampled parameter values
params = np.genfromtxt(''.join([path, '/data/LHC_samples_N1000_DIMS5.csv']), delimiter=',')

# pull data from diff files 
ssp5_paths_and_util = np.genfromtxt(''.join([path, '/data/lhc_sampling_paths_ssp5_N1000.csv']), delimiter=',')
ssp5_exp_m = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_m_ssp5_N1000.csv']), delimiter=',')
ssp5_exp_t = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_T_ssp5_N1000.csv']), delimiter=',')
ssp5_exp_co2 = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_conc_ssp5_N1000.csv']), delimiter=',')
ssp5_exp_dam = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_dam_ssp5_N1000.csv']), delimiter=',')
ssp5_paths, ssp5_c2020, ssp5_c2030, ssp5_c2060, ssp5_c2100, ssp5_c2150, ssp5_c2200, ssp5_util,\
    ssp5_params, ssp5_exp_m, ssp5_exp_t, ssp5_exp_co2, ssp5_exp_dam = remove_nans(ssp5_paths_and_util, 
                                                                                  ssp5_exp_m, ssp5_exp_t, 
                                                                                  ssp5_exp_co2, ssp5_exp_dam, 
                                                                                  params)
ssp5_ra, ssp5_eis, ssp5_ex, ssp5_end, ssp5_prtp = ssp5_params.T

ssp4_paths_and_util = np.genfromtxt(''.join([path, '/data/lhc_sampling_paths_ssp4_N1000.csv']), delimiter=',')
ssp4_exp_m = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_m_ssp4_N1000.csv']), delimiter=',')
ssp4_exp_t = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_T_ssp4_N1000.csv']), delimiter=',')
ssp4_exp_co2 = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_conc_ssp4_N1000.csv']), delimiter=',')
ssp4_exp_dam = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_dam_ssp4_N1000.csv']), delimiter=',')
ssp4_paths, ssp4_c2020, ssp4_c2030, ssp4_c2060, ssp4_c2100, ssp4_c2150, ssp4_c2200, ssp4_util,\
    ssp4_params, ssp4_exp_m, ssp4_exp_t, ssp4_exp_co2, ssp4_exp_dam = remove_nans(ssp4_paths_and_util, 
                                                                                  ssp4_exp_m, ssp4_exp_t, 
                                                                                  ssp4_exp_co2, ssp4_exp_dam, 
                                                                                  params)
ssp4_ra, ssp4_eis, ssp4_ex, ssp4_end, ssp4_prtp = ssp4_params.T

ssp3_paths_and_util = np.genfromtxt(''.join([path, '/data/lhc_sampling_paths_ssp3_N1000.csv']), delimiter=',')
ssp3_exp_m = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_m_ssp3_N1000.csv']), delimiter=',')
ssp3_exp_t = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_T_ssp3_N1000.csv']), delimiter=',')
ssp3_exp_co2 = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_conc_ssp3_N1000.csv']), delimiter=',')
ssp3_exp_dam = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_dam_ssp3_N1000.csv']), delimiter=',')
ssp3_paths, ssp3_c2020, ssp3_c2030, ssp3_c2060, ssp3_c2100, ssp3_c2150, ssp3_c2200, ssp3_util,\
    ssp3_params, ssp3_exp_m, ssp3_exp_t, ssp3_exp_co2, ssp3_exp_dam = remove_nans(ssp3_paths_and_util, 
                                                                                  ssp3_exp_m, ssp3_exp_t, 
                                                                                  ssp3_exp_co2, ssp3_exp_dam, 
                                                                                  params)
ssp3_ra, ssp3_eis, ssp3_ex, ssp3_end, ssp3_prtp = ssp3_params.T

ssp2_paths_and_util = np.genfromtxt(''.join([path, '/data/lhc_sampling_paths_ssp2_N1000.csv']), delimiter=',')
ssp2_exp_m = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_m_ssp2_N1000.csv']), delimiter=',')
ssp2_exp_t = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_T_ssp2_N1000.csv']), delimiter=',')
ssp2_exp_co2 = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_conc_ssp2_N1000.csv']), delimiter=',')
ssp2_exp_dam = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_dam_ssp2_N1000.csv']), delimiter=',')
ssp2_paths, ssp2_c2020, ssp2_c2030, ssp2_c2060, ssp2_c2100, ssp2_c2150, ssp2_c2200, ssp2_util,\
    ssp2_params, ssp2_exp_m, ssp2_exp_t, ssp2_exp_co2, ssp2_exp_dam = remove_nans(ssp2_paths_and_util, 
                                                                                  ssp2_exp_m, ssp2_exp_t, 
                                                                                  ssp2_exp_co2, ssp2_exp_dam, 
                                                                                  params)
ssp2_ra, ssp2_eis, ssp2_ex, ssp2_end, ssp2_prtp = ssp2_params.T

ssp1_paths_and_util = np.genfromtxt(''.join([path, '/data/lhc_sampling_paths_ssp1_N1000.csv']), delimiter=',')
ssp1_exp_m = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_m_ssp1_N1000.csv']), delimiter=',')
ssp1_exp_t = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_T_ssp1_N1000.csv']), delimiter=',')
ssp1_exp_co2 = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_conc_ssp1_N1000.csv']), delimiter=',')
ssp1_exp_dam = np.genfromtxt(''.join([path, '/data/lhc_sampling_exp_dam_ssp1_N1000.csv']), delimiter=',')
ssp1_paths, ssp1_c2020, ssp1_c2030, ssp1_c2060, ssp1_c2100, ssp1_c2150, ssp1_c2200, ssp1_util,\
    ssp1_params, ssp1_exp_m, ssp1_exp_t, ssp1_exp_co2, ssp1_exp_dam = remove_nans(ssp1_paths_and_util, 
                                                                                  ssp1_exp_m, ssp1_exp_t, 
                                                                                  ssp1_exp_co2, ssp1_exp_dam, 
                                                                                  params)
ssp1_ra, ssp1_eis, ssp1_ex, ssp1_end, ssp1_prtp = ssp1_params.T

## Unpack temperature, CO<sub>2</sub> concentrations and damages data

In [ ]:
ssp5_t2020, ssp5_t2030, ssp5_t2060, ssp5_t2100, ssp5_t2150, ssp5_t2200 = ssp5_exp_t.T
ssp5_co2020, ssp5_co2030, ssp5_co2060, ssp5_co2100, ssp5_co2150, ssp5_co2200 = ssp5_exp_co2.T
ssp5_d2020, ssp5_d2030, ssp5_d2060, ssp5_d2100, ssp5_d2150, ssp5_d2200 = ssp5_exp_dam.T

ssp4_t2020, ssp4_t2030, ssp4_t2060, ssp4_t2100, ssp4_t2150, ssp4_t2200 = ssp4_exp_t.T
ssp4_co2020, ssp4_co2030, ssp4_co2060, ssp4_co2100, ssp4_co2150, ssp4_co2200 = ssp4_exp_co2.T
ssp4_d2020, ssp4_d2030, ssp4_d2060, ssp4_d2100, ssp4_d2150, ssp4_d2200 = ssp4_exp_dam.T

ssp3_t2020, ssp3_t2030, ssp3_t2060, ssp3_t2100, ssp3_t2150, ssp3_t2200 = ssp3_exp_t.T
ssp3_co2020, ssp3_co2030, ssp3_co2060, ssp3_co2100, ssp3_co2150, ssp3_co2200 = ssp3_exp_co2.T
ssp3_d2020, ssp3_d2030, ssp3_d2060, ssp3_d2100, ssp3_d2150, ssp3_d2200 = ssp3_exp_dam.T

ssp2_t2020, ssp2_t2030, ssp2_t2060, ssp2_t2100, ssp2_t2150, ssp2_t2200 = ssp2_exp_t.T
ssp2_co2020, ssp2_co2030, ssp2_co2060, ssp2_co2100, ssp2_co2150, ssp2_co2200 = ssp2_exp_co2.T
ssp2_d2020, ssp2_d2030, ssp2_d2060, ssp2_d2100, ssp2_d2150, ssp2_d2200 = ssp2_exp_dam.T

ssp1_t2020, ssp1_t2030, ssp1_t2060, ssp1_t2100, ssp1_t2150, ssp1_t2200 = ssp1_exp_t.T
ssp1_co2020, ssp1_co2030, ssp1_co2060, ssp1_co2100, ssp1_co2150, ssp1_co2200 = ssp1_exp_co2.T
ssp1_d2020, ssp1_d2030, ssp1_d2060, ssp1_d2100, ssp1_d2150, ssp1_d2200 = ssp1_exp_dam.T

## Make scatter and regression plot

Scatter plots of price vs parameter at every point in time with regression.

### Combine tech change into one parameter: $\varphi_{t} := \varphi_{0} + \varphi_{1} X_{t}$

In [ ]:
# Making total tech change parameter

ssp5_tottech = ssp5_ex[:, None] + ssp5_end[:, None] * ssp5_exp_m
ssp4_tottech = ssp4_ex[:, None] + ssp4_end[:, None] * ssp4_exp_m
ssp3_tottech = ssp3_ex[:, None] + ssp3_end[:, None] * ssp3_exp_m
ssp2_tottech = ssp2_ex[:, None] + ssp2_end[:, None] * ssp2_exp_m
ssp1_tottech = ssp1_ex[:, None] + ssp1_end[:, None] * ssp1_exp_m

### Make functions for plotting

It's quite a lot, as I did a lot of different tests to see which regressions worked best. 

#### Basic linear regression panel

In [ ]:
from sklearn.linear_model import LinearRegression

def get_regression(param, costs):
    LR = LinearRegression()
    reg = LR.fit(param.reshape(-1,1), costs)
    a, b = reg.coef_[0], reg.intercept_
    score = reg.score(param.reshape(-1,1), costs)
    return a, b, score

def make_panel_w_regression(ax, grid_x, grid_y, ssp5_param, ssp4_param, ssp3_param, ssp2_param, ssp1_param, ssp5_prices, ssp4_prices, ssp3_prices, ssp2_prices, ssp1_prices,
              x_label, y_label):
    ssp5_a, ssp5_b, ssp5_r2 = get_regression(ssp5_param, ssp5_prices)
    ssp4_a, ssp4_b, ssp4_r2 = get_regression(ssp4_param, ssp4_prices)
    ssp3_a, ssp3_b, ssp3_r2 = get_regression(ssp3_param, ssp3_prices)
    ssp2_a, ssp2_b, ssp2_r2 = get_regression(ssp2_param, ssp2_prices)
    ssp1_a, ssp1_b, ssp1_r2 = get_regression(ssp1_param, ssp1_prices)
    
    ssp5_param_range = np.arange(min(ssp5_param), max(ssp5_param), 0.01)
    ssp4_param_range = np.arange(min(ssp4_param), max(ssp4_param), 0.01)
    ssp3_param_range = np.arange(min(ssp3_param), max(ssp3_param), 0.01)
    ssp2_param_range = np.arange(min(ssp2_param), max(ssp2_param), 0.01)
    ssp1_param_range = np.arange(min(ssp1_param), max(ssp1_param), 0.01)
    
    alpha = 0.2
    ax[grid_x, grid_y].scatter(ssp5_param, ssp5_prices, c=color_list[ssp5_cind], marker=marker_list[0], alpha=alpha)
    ax[grid_x, grid_y].scatter(ssp4_param, ssp4_prices, c=color_list[ssp4_cind], marker=marker_list[1], alpha=alpha)
    ax[grid_x, grid_y].scatter(ssp3_param, ssp3_prices, c=color_list[ssp3_cind], marker=marker_list[2], alpha=alpha)
    ax[grid_x, grid_y].scatter(ssp2_param, ssp2_prices, c=color_list[ssp2_cind], marker=marker_list[3], alpha=alpha)
    ax[grid_x, grid_y].scatter(ssp1_param, ssp1_prices, c=color_list[ssp1_cind], marker=marker_list[4], alpha=alpha)
    
    ax[grid_x, grid_y].plot(ssp5_param_range, ssp5_a * ssp5_param_range + ssp5_b, c=color_list[ssp5_cind],
                            linewidth=3, label=''.join(["SSP5, r$^{2} =$", "{:0.2f}".format(ssp5_r2)]))
    ax[grid_x, grid_y].plot(ssp4_param_range, ssp4_a * ssp4_param_range + ssp4_b, c=color_list[ssp4_cind],
                            linewidth=3, label=''.join(["SSP4, r$^{2} =$", "{:0.2f}".format(ssp4_r2)]))
    ax[grid_x, grid_y].plot(ssp3_param_range, ssp3_a * ssp3_param_range + ssp3_b, c=color_list[ssp3_cind],
                            linewidth=3, label=''.join(["SSP3, r$^{2} =$", "{:0.2f}".format(ssp3_r2)]))
    ax[grid_x, grid_y].plot(ssp2_param_range, ssp2_a * ssp2_param_range + ssp2_b, c=color_list[ssp2_cind],
                            linewidth=3, label=''.join(["SSP2, r$^{2} =$", "{:0.2f}".format(ssp2_r2)]))
    ax[grid_x, grid_y].plot(ssp1_param_range, ssp1_a * ssp1_param_range + ssp1_b, c=color_list[ssp1_cind],
                            linewidth=3, label=''.join(["SSP1, r$^{2} =$", "{:0.2f}".format(ssp1_r2)]))
    
    ax[grid_x, grid_y].legend(fontsize=15, frameon=True, facecolor='white', framealpha=0.9)
    ax[grid_x, grid_y].set_xlabel(x_label)
    ax[grid_x, grid_y].set_ylabel(y_label)
    if x_label == "Risk Aversion":
        ax[grid_x, grid_y].set_xticks([3, 9, 15])
    elif x_label == "EIS":
        ax[grid_x, grid_y].set_xticks([0.5, 0.8, 1.1])
    elif x_label == r"$\delta$ (%)":
        ax[grid_x, grid_y].set_xticks([0.1, 0.45, 0.8, 1.15, 1.5])

#### Quadratic regression panel

In [ ]:
from sklearn.metrics import r2_score

def get_quad_regression(param, costs):
    a2, a1, a0 = np.polyfit(param, costs, deg=2)
    r2 = r2_score(costs, a2 * param**2 + a1 * param + a0)
    return a2, a1, a0, r2

def make_panel_w_quad_regression(ax, grid_x, grid_y, ssp5_param, ssp4_param, ssp3_param, ssp2_param, ssp1_param, ssp5_prices, ssp4_prices, ssp3_prices, ssp2_prices, ssp1_prices,
              x_label, y_label):
    ssp5_a2, ssp5_a1, ssp5_a0, ssp5_r2 = get_quad_regression(ssp5_param, ssp5_prices)
    ssp4_a2, ssp4_a1, ssp4_a0, ssp4_r2 = get_quad_regression(ssp4_param, ssp4_prices)
    ssp3_a2, ssp3_a1, ssp3_a0, ssp3_r2 = get_quad_regression(ssp3_param, ssp3_prices)
    ssp2_a2, ssp2_a1, ssp2_a0, ssp2_r2 = get_quad_regression(ssp2_param, ssp2_prices)
    ssp1_a2, ssp1_a1, ssp1_a0, ssp1_r2 = get_quad_regression(ssp1_param, ssp1_prices)
    
    ssp5_param_range = np.arange(min(ssp5_param), max(ssp5_param), 0.01)
    ssp4_param_range = np.arange(min(ssp4_param), max(ssp4_param), 0.01)
    ssp3_param_range = np.arange(min(ssp3_param), max(ssp3_param), 0.01)
    ssp2_param_range = np.arange(min(ssp2_param), max(ssp2_param), 0.01)
    ssp1_param_range = np.arange(min(ssp1_param), max(ssp1_param), 0.01)
    
    alpha = 0.2
    ax[grid_x, grid_y].scatter(ssp5_param, ssp5_prices, c=color_list[ssp5_cind], marker=marker_list[0], alpha=alpha)
    ax[grid_x, grid_y].scatter(ssp4_param, ssp4_prices, c=color_list[ssp4_cind], marker=marker_list[1], alpha=alpha)
    ax[grid_x, grid_y].scatter(ssp3_param, ssp3_prices, c=color_list[ssp3_cind], marker=marker_list[2], alpha=alpha)
    ax[grid_x, grid_y].scatter(ssp2_param, ssp2_prices, c=color_list[ssp2_cind], marker=marker_list[3], alpha=alpha)
    ax[grid_x, grid_y].scatter(ssp1_param, ssp1_prices, c=color_list[ssp1_cind], marker=marker_list[4], alpha=alpha)
    
    ax[grid_x, grid_y].plot(ssp5_param_range, ssp5_a2 * ssp5_param_range**2 + ssp5_a1 * ssp5_param_range + ssp5_a0, c=color_list[ssp5_cind],
                            linewidth=3, label=''.join(["SSP5, r$^{2} =$", "{:0.2f}".format(ssp5_r2)]))
    ax[grid_x, grid_y].plot(ssp4_param_range, ssp4_a2 * ssp4_param_range**2 + ssp4_a1 * ssp4_param_range + ssp4_a0, c=color_list[ssp4_cind],
                            linewidth=3, label=''.join(["SSP4, r$^{2} =$", "{:0.2f}".format(ssp4_r2)]))
    ax[grid_x, grid_y].plot(ssp3_param_range, ssp3_a2 * ssp3_param_range**2 + ssp3_a1 * ssp3_param_range + ssp3_a0, c=color_list[ssp3_cind],
                            linewidth=3, label=''.join(["SSP3, r$^{2} =$", "{:0.2f}".format(ssp3_r2)]))
    ax[grid_x, grid_y].plot(ssp2_param_range, ssp2_a2 * ssp2_param_range**2 + ssp2_a1 * ssp2_param_range + ssp2_a0, c=color_list[ssp2_cind],
                            linewidth=3, label=''.join(["SSP2, r$^{2} =$", "{:0.2f}".format(ssp2_r2)]))
    ax[grid_x, grid_y].plot(ssp1_param_range, ssp1_a2 * ssp1_param_range**2 + ssp1_a1 * ssp1_param_range + ssp1_a0, c=color_list[ssp1_cind],
                            linewidth=3, label=''.join(["SSP1, r$^{2} =$", "{:0.2f}".format(ssp1_r2)]))
    
    ax[grid_x, grid_y].legend(fontsize=15, frameon=True, facecolor='white', framealpha=0.9)
    ax[grid_x, grid_y].set_xlabel(x_label)
    ax[grid_x, grid_y].set_ylabel(y_label)

### Make plot!

In [ ]:
fig, ax = plt.subplots(4, 6, figsize=(30, 20))

ax[0,0].set_title("2020")
ax[0,1].set_title("2030")
ax[0,2].set_title("2060")
ax[0,3].set_title("2100")
ax[0,4].set_title("2150")
ax[0,5].set_title("2200")

###########################
## RISK AVERSION ROW
###########################
make_panel_w_regression(ax, 0, 0, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2020, ssp4_c2020, ssp3_c2020, ssp2_c2020, ssp1_c2020, 
           "Risk Aversion", "Cost\n(2020 $USD)")

make_panel_w_regression(ax, 0, 1, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2030, ssp4_c2030, ssp3_c2030, ssp2_c2030, ssp1_c2030, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 2, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2060, ssp4_c2060, ssp3_c2060, ssp2_c2060, ssp1_c2060, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 3, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2100, ssp4_c2100, ssp3_c2100, ssp2_c2100, ssp1_c2100, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 4, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2150, ssp4_c2150, ssp3_c2150, ssp2_c2150, ssp1_c2150, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 5, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2200, ssp4_c2200, ssp3_c2200, ssp2_c2200, ssp1_c2200, 
           "Risk Aversion", None)

###########################
# EIS ROW
###########################
make_panel_w_regression(ax, 1, 0, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2020, ssp4_c2020, ssp3_c2020, ssp2_c2020, ssp1_c2020, 
           "EIS", "Cost\n(2020 $USD)")

make_panel_w_regression(ax, 1, 1, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2030, ssp4_c2030, ssp3_c2030, ssp2_c2030, ssp1_c2030, 
           "EIS", None)

make_panel_w_regression(ax, 1, 2, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2060, ssp4_c2060, ssp3_c2060, ssp2_c2060, ssp1_c2060, 
           "EIS", None)

make_panel_w_regression(ax, 1, 3, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2100, ssp4_c2100, ssp3_c2100, ssp2_c2100, ssp1_c2100, 
           "EIS", None)

make_panel_w_regression(ax, 1, 4, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2150, ssp4_c2150, ssp3_c2150, ssp2_c2150, ssp1_c2150, 
           "EIS", None)

make_panel_w_regression(ax, 1, 5, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2200, ssp4_c2200, ssp3_c2200, ssp2_c2200, ssp1_c2200, 
           "EIS", None)

###########################
# PRTP ROW
###########################
make_panel_w_regression(ax, 2, 0, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2020, ssp4_c2020, ssp3_c2020, ssp2_c2020, ssp1_c2020, 
           r"$\delta$ (%)", "Cost\n(2020 $USD)")

make_panel_w_regression(ax, 2, 1, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2030, ssp4_c2030, ssp3_c2030, ssp2_c2030, ssp1_c2030, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 2, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2060, ssp4_c2060, ssp3_c2060, ssp2_c2060, ssp1_c2060, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 3, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2100, ssp4_c2100, ssp3_c2100, ssp2_c2100, ssp1_c2100, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 4, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2150, ssp4_c2150, ssp3_c2150, ssp2_c2150, ssp1_c2150, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 5, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2200, ssp4_c2200, ssp3_c2200, ssp2_c2200, ssp1_c2200, 
           r"$\delta$ (%)", None)

###########################
# EX TECH GROWTH ROW
###########################
make_panel_w_regression(ax, 3, 0, ssp5_tottech[:, 0], ssp4_tottech[:, 0], ssp3_tottech[:, 0],
                        ssp2_tottech[:, 0], ssp1_tottech[:, 0], ssp5_c2020, ssp4_c2020, ssp3_c2020, 
                        ssp2_c2020, ssp1_c2020, "Tech Growth (%)", "Cost\n(2020 $USD)")

make_panel_w_regression(ax, 3, 1, ssp5_tottech[:, 1], ssp4_tottech[:, 1], ssp3_tottech[:, 1], 
                        ssp2_tottech[:, 1], ssp1_tottech[:, 1], ssp5_c2030, ssp4_c2030, ssp3_c2030, 
                        ssp2_c2030, ssp1_c2030, "Tech Growth (%)", None)

make_panel_w_quad_regression(ax, 3, 2, ssp5_tottech[:, 2], ssp4_tottech[:, 2], ssp3_tottech[:, 2], 
                        ssp2_tottech[:, 2], ssp1_tottech[:, 2], ssp5_c2060, ssp4_c2060, ssp3_c2060, 
                        ssp2_c2060, ssp1_c2060, "Tech Growth (%)", None)

make_panel_w_quad_regression(ax, 3, 3, ssp5_tottech[:, 3], ssp4_tottech[:, 3], ssp3_tottech[:, 3], 
                        ssp2_tottech[:, 3], ssp1_tottech[:, 3], ssp5_c2100, ssp4_c2100, ssp3_c2100, 
                        ssp2_c2100, ssp1_c2100, "Tech Growth (%)", None)

make_panel_w_quad_regression(ax, 3, 4, ssp5_tottech[:, 4], ssp4_tottech[:, 4], ssp3_tottech[:, 4], 
                        ssp2_tottech[:, 4], ssp1_tottech[:, 4], ssp5_c2150, ssp4_c2150, ssp3_c2150, 
                        ssp2_c2150, ssp1_c2150, "Tech Growth (%)", None)

make_panel_w_quad_regression(ax, 3, 5, ssp5_tottech[:, 5], ssp4_tottech[:, 5], ssp3_tottech[:, 5], 
                        ssp2_tottech[:, 5], ssp1_tottech[:, 5], ssp5_c2200, ssp4_c2200, ssp3_c2200, 
                        ssp2_c2200, ssp1_c2200, "Tech Growth (%)", None)

ax[3,0].set_xticks([0,1,2,3])
ax[3,0].set_xlim((0,3))
ax[3,1].set_xticks([0,1,2,3,4,5])
ax[3,2].set_xticks([0,1,2,3,4,5,6])
ax[3,3].set_xticks([0,1,2,3,4,5,6])
ax[3,4].set_xticks([0,1,2,3,4,5,6])
ax[3,5].set_xticks([0,1,2,3,4,5,6])

fig.tight_layout()
sns.despine(trim=True, offset=10)

if save_fig:
    fig.savefig(''.join([basefile, 'regression-sticky-cost.png']), dpi=400)    

## Parition $r^2$ sources

We now partition the variance into different parameters using the $r^2$ calculated above.

### $r^2$ curves in time functions

The first is for all parameters that aren't tech change. Since tech change required two regressions in the later periods, it needs its own function where the $r^2$ of each curve is averaged and reported as the $r^2$ of that period.

In [ ]:
# regressions

def get_r2_in_time(param, paths):
    c2020, c2030, c2060, c2100, c2150, c2200 = paths.T
    _, _, r2_2020 = get_regression(param, c2020)
    _, _, r2_2030 = get_regression(param, c2030)
    _, _, r2_2060 = get_regression(param, c2060)
    _, _, r2_2100 = get_regression(param, c2100)
    _, _, r2_2150 = get_regression(param, c2150)
    _, _, r2_2200 = get_regression(param, c2200)
    
    r2 = np.array([r2_2020, r2_2030, r2_2060, r2_2100, r2_2150, r2_2200])
    return r2

In [ ]:
def get_tech_r2_in_time(tottech, paths, bp1_2150, bp2_2150, bp1_2200, bp2_2200, ssp1):
    c2020, c2030, c2060, c2100, c2150, c2200 = paths.T
    _, _, r2_2020 = get_regression(tottech[:,0], c2020)
    _, _, r2_2030 = get_regression(tottech[:,1], c2030)
    _, _, _, r2_2060 = get_quad_regression(tottech[:,2], c2060)
    _, _, _, r2_2100 = get_quad_regression(tottech[:,3], c2100)
    
    ssp5_param_1, ssp5_param_2, ssp5_param_3, ssp5_prices_1, ssp5_prices_2, ssp5_prices_3 = get_double_partition(tottech[:,5], c2150, bp1_2150, bp2_2150)
    
    _, _, _, ssp5_r2_1 = get_quad_regression(ssp5_param_1, ssp5_prices_1)
        
    _, _, _, ssp5_r2_3 = get_quad_regression(ssp5_param_3, ssp5_prices_3)

    if ssp1:
        r2_2150 = np.mean([ssp5_r2_1])
    else:
        r2_2150 = np.mean([ssp5_r2_1, ssp5_r2_3])
    
    ssp5_param_1, ssp5_param_2, ssp5_param_3, ssp5_prices_1, ssp5_prices_2, ssp5_prices_3 = get_double_partition(tottech[:,5], c2200, bp2_2200, bp2_2200)
    
    _, _, _, ssp5_r2_1 = get_quad_regression(ssp5_param_1, ssp5_prices_1)
        
    _, _, _, ssp5_r2_3 = get_quad_regression(ssp5_param_3, ssp5_prices_3)

    r2_2200 = np.mean([ssp5_r2_1, ssp5_r2_3])
    
    r2 = np.array([r2_2020, r2_2030, r2_2060, r2_2100, r2_2150, r2_2200])
    return r2

In [ ]:
ssp5_eis_r2 = get_r2_in_time(ssp5_eis, ssp5_paths)
ssp5_ra_r2 = get_r2_in_time(ssp5_ra, ssp5_paths)
ssp5_prtp_r2 = get_r2_in_time(ssp5_prtp, ssp5_paths)
ssp5_tech_r2 = get_tech_r2_in_time(ssp5_tottech, ssp5_paths, bp1s_2150[0], bp2s_2150[0], 
                                   bp1s_2200[0], bp2s_2200[0], False)

ssp4_eis_r2 = get_r2_in_time(ssp4_eis, ssp4_paths)
ssp4_ra_r2 = get_r2_in_time(ssp4_ra, ssp4_paths)
ssp4_prtp_r2 = get_r2_in_time(ssp4_prtp, ssp4_paths)
ssp4_tech_r2 = get_tech_r2_in_time(ssp4_tottech, ssp4_paths, bp1s_2150[1], bp2s_2150[1], 
                                   bp1s_2200[1], bp2s_2200[1], False)

ssp3_eis_r2 = get_r2_in_time(ssp3_eis, ssp3_paths)
ssp3_ra_r2 = get_r2_in_time(ssp3_ra, ssp3_paths)
ssp3_prtp_r2 = get_r2_in_time(ssp3_prtp, ssp3_paths)
ssp3_tech_r2 = get_tech_r2_in_time(ssp3_tottech, ssp3_paths, bp1s_2150[2], bp2s_2150[2], 
                                   bp1s_2200[2], bp2s_2200[2], False)

ssp2_eis_r2 = get_r2_in_time(ssp2_eis, ssp2_paths)
ssp2_ra_r2 = get_r2_in_time(ssp2_ra, ssp2_paths)
ssp2_prtp_r2 = get_r2_in_time(ssp2_prtp, ssp2_paths)
ssp2_tech_r2 = get_tech_r2_in_time(ssp2_tottech, ssp2_paths, bp1s_2150[3], bp2s_2150[3], 
                                   bp1s_2200[3], bp2s_2200[3], False)

ssp1_eis_r2 = get_r2_in_time(ssp1_eis, ssp1_paths)
ssp1_ra_r2 = get_r2_in_time(ssp1_ra, ssp1_paths)
ssp1_prtp_r2 = get_r2_in_time(ssp1_prtp, ssp1_paths)
ssp1_tech_r2 = get_tech_r2_in_time(ssp1_tottech, ssp1_paths, bp1s_2150[4], bp2s_2150[4], 
                                   bp1s_2200[4], bp2s_2200[4], True)

## Make plot of raw $r^2$ values

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(20,20), sharey=True)

ax[0].set_ylim((0,1))

alpha = 0.8
lw = 3
l1cind = 0
l2cind = 1
l3cind = 2
l4cind = 3
l5cind = 4

################################
# SSP5
################################
ax[0].plot(node_times, ssp5_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[0].fill_between(node_times, np.zeros_like(node_times), ssp5_ra_r2, alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[0].plot(node_times, ssp5_eis_r2 + ssp5_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[0].fill_between(node_times, ssp5_ra_r2, ssp5_ra_r2 + ssp5_eis_r2, alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[0].plot(node_times, ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[0].fill_between(node_times, ssp5_eis_r2 + ssp5_ra_r2, ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2, alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[0].plot(node_times, ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2 + ssp5_tech_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[0].fill_between(node_times, ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2, ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2 + ssp5_tech_r2, alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
ax[1].plot(node_times, ssp4_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[1].fill_between(node_times, np.zeros_like(node_times), ssp4_ra_r2, alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[1].plot(node_times, ssp4_eis_r2 + ssp4_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[1].fill_between(node_times, ssp4_ra_r2, ssp5_ra_r2 + ssp4_eis_r2, alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[1].plot(node_times, ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[1].fill_between(node_times, ssp4_eis_r2 + ssp4_ra_r2, ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2, alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[1].plot(node_times, ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2 + ssp4_tech_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[1].fill_between(node_times, ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2, ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2 + ssp4_tech_r2, alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ax[2].plot(node_times, ssp3_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[2].fill_between(node_times, np.zeros_like(node_times), ssp3_ra_r2, alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[2].plot(node_times, ssp3_eis_r2 + ssp3_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[2].fill_between(node_times, ssp3_ra_r2, ssp3_ra_r2 + ssp3_eis_r2, alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[2].plot(node_times, ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[2].fill_between(node_times, ssp3_eis_r2 + ssp3_ra_r2, ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2, alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[2].plot(node_times, ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2 + ssp3_tech_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[2].fill_between(node_times, ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2, ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2 + ssp3_tech_r2, alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ax[3].plot(node_times, ssp2_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[3].fill_between(node_times, np.zeros_like(node_times), ssp2_ra_r2, alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[3].plot(node_times, ssp2_eis_r2 + ssp2_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[3].fill_between(node_times, ssp2_ra_r2, ssp2_ra_r2 + ssp2_eis_r2, alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[3].plot(node_times, ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[3].fill_between(node_times, ssp2_eis_r2 + ssp2_ra_r2, ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2, alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[3].plot(node_times, ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2 + ssp2_tech_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[3].fill_between(node_times, ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2, ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2 + ssp2_tech_r2, alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ax[4].plot(node_times, ssp1_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[4].fill_between(node_times, np.zeros_like(node_times), ssp5_ra_r2, alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[4].plot(node_times, ssp1_eis_r2 + ssp1_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[4].fill_between(node_times, ssp1_ra_r2, ssp1_ra_r2 + ssp1_eis_r2, alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[4].plot(node_times, ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[4].fill_between(node_times, ssp1_eis_r2 + ssp1_ra_r2, ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2, alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[4].plot(node_times, ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2 + ssp1_tech_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[4].fill_between(node_times, ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2, ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2 + ssp1_tech_r2, alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

ax[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=5)

ax[0].set_ylabel("r$^{2}$")
ax[0].set_title("SSP5")
ax[1].set_title("SSP4")
ax[2].set_title("SSP3")
ax[3].set_title("SSP2")
ax[4].set_title("SSP1")

for i in range(5):
    ax[i].set_xlim((node_times[0], node_times[-1]))
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_xlabel("Year")

#fig.tight_layout()

## Make plot using fraction of available $r^2$

The missing $r^2$ in the above can be chalked up to either optimization variance or other variance (like in damage calculations, etc., but we expect the majority to be optimization error).

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(20,20), sharey=True)

ax[0].set_ylim((0,1))

alpha = 0.8
lw = 3
l1cind = 0
l2cind = 1
l3cind = 2
l4cind = 3
l5cind = 4

################################
# SSP5
################################
tot_ssp5 = ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2 + ssp5_tech_r2
ax[0].plot(node_times, ssp5_ra_r2 * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[0].fill_between(node_times, np.zeros_like(node_times), ssp5_ra_r2 * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[0].plot(node_times, (ssp5_eis_r2 + ssp5_ra_r2)  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[0].fill_between(node_times, ssp5_ra_r2  * tot_ssp5**(-1), 
                   (ssp5_ra_r2 + ssp5_eis_r2) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[0].plot(node_times, (ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2)  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[0].fill_between(node_times, (ssp5_eis_r2 + ssp5_ra_r2) * tot_ssp5**(-1), 
                  (ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[0].plot(node_times, (ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2 + ssp5_tech_r2) * tot_ssp5**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax[0].fill_between(node_times, (ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2) * tot_ssp5**(-1), 
                   (ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2 + ssp5_tech_r2) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
tot_ssp4 = ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2 + ssp4_tech_r2
ax[1].plot(node_times, ssp4_ra_r2 * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[1].fill_between(node_times, np.zeros_like(node_times), ssp4_ra_r2 * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[1].plot(node_times, (ssp4_eis_r2 + ssp4_ra_r2) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[1].fill_between(node_times, ssp4_ra_r2 * tot_ssp4**(-1), 
                   (ssp4_ra_r2 + ssp4_eis_r2) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[1].plot(node_times, (ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[1].fill_between(node_times, (ssp4_eis_r2 + ssp4_ra_r2) * tot_ssp4**(-1), 
                  (ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[1].plot(node_times, (ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2 + ssp4_tech_r2) * tot_ssp4**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax[1].fill_between(node_times, (ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2) * tot_ssp4**(-1), 
                   (ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2 + ssp4_tech_r2) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ssp3_tot = ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2 + ssp3_tech_r2
ax[2].plot(node_times, ssp3_ra_r2 * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[2].fill_between(node_times, np.zeros_like(node_times), ssp3_ra_r2 * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[2].plot(node_times, (ssp3_eis_r2 + ssp3_ra_r2) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[2].fill_between(node_times, ssp3_ra_r2 * ssp3_tot**(-1), 
                   (ssp3_ra_r2 + ssp3_eis_r2) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[2].plot(node_times, (ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[2].fill_between(node_times, (ssp3_eis_r2 + ssp3_ra_r2) * ssp3_tot**(-1), 
                   (ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[2].plot(node_times, (ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2 + ssp3_tech_r2) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[2].fill_between(node_times, (ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2) * ssp3_tot**(-1), 
                   (ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2 + ssp3_tech_r2) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ssp2_tot = ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2 + ssp2_tech_r2
ax[3].plot(node_times, ssp2_ra_r2 * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[3].fill_between(node_times, np.zeros_like(node_times), ssp2_ra_r2* ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[3].plot(node_times, (ssp2_eis_r2 + ssp2_ra_r2)* ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[3].fill_between(node_times, ssp2_ra_r2 * ssp2_tot**(-1), 
                   (ssp2_ra_r2 + ssp2_eis_r2) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[3].plot(node_times, (ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[3].fill_between(node_times, (ssp2_eis_r2 + ssp2_ra_r2) * ssp2_tot**(-1), 
                   (ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[3].plot(node_times, (ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2 + ssp2_tech_r2) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[3].fill_between(node_times, (ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2) * ssp2_tot**(-1), 
                   (ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2 + ssp2_tech_r2) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ssp1_tot = ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2 + ssp1_tech_r2
ax[4].plot(node_times, ssp1_ra_r2 * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[4].fill_between(node_times, np.zeros_like(node_times), ssp5_ra_r2 * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[4].plot(node_times, (ssp1_eis_r2 + ssp1_ra_r2) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[4].fill_between(node_times, ssp1_ra_r2 * ssp1_tot**(-1), 
                   (ssp1_ra_r2 + ssp1_eis_r2) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l2cind], label="EIS")

ax[4].plot(node_times, (ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[4].fill_between(node_times, (ssp1_eis_r2 + ssp1_ra_r2) * ssp1_tot**(-1), 
                   (ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l3cind], label="PRTP")

ax[4].plot(node_times, (ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2 + ssp1_tech_r2) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[4].fill_between(node_times, (ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2) * ssp1_tot**(-1), 
                   (ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2 + ssp1_tech_r2) * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

ax[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=5, fontsize=20)

ax[0].set_ylabel("$r^{2} / r_{tot}^{2}$")
ax[0].set_title("SSP5")
ax[1].set_title("SSP4")
ax[2].set_title("SSP3")
ax[3].set_title("SSP2")
ax[4].set_title("SSP1")

panel_label = ["A", "B", "C", "D", "E"]
for i in range(5):
    ax[i].set_xlim((node_times[0], node_times[-1]))
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_xlabel("Year")
    ax[i].text(2168, 0.88, panel_label[i], fontsize=22, color='white', fontweight='bold')
    ax[i].set_xticks([2020,2100,2200])
    ax[i].set_xticklabels(['2020', '2100', '2200'])
    ax[i].tick_params(axis='both', labelsize=14)

sns.despine(trim=True, offset=10)

if save_fig:
    fig.savefig(''.join([basefile, 'rsquared.png']), dpi=400)

# And now, temperature

In [ ]:
fig, ax = plt.subplots(4, 5, figsize=(30, 20))

ax[0,0].set_title("2030")
ax[0,1].set_title("2060")
ax[0,2].set_title("2100")
ax[0,3].set_title("2150")
ax[0,4].set_title("2200")

###########################
## RISK AVERSION ROW
###########################
make_panel_w_regression(ax, 0, 0, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_t2030, ssp4_t2030, ssp3_t2030, ssp2_t2030, ssp1_t2030, 
           "Risk Aversion", "Temperature ($^{\circ}$C)")

make_panel_w_regression(ax, 0, 1, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_t2060, ssp4_t2060, ssp3_t2060, ssp2_t2060, ssp1_t2060, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 2, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_t2100, ssp4_t2100, ssp3_t2100, ssp2_t2100, ssp1_t2100, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 3, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_t2150, ssp4_t2150, ssp3_t2150, ssp2_t2150, ssp1_t2150, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 4, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_t2200, ssp4_t2200, ssp3_t2200, ssp2_t2200, ssp1_t2200, 
           "Risk Aversion", None)

###########################
# EIS ROW
###########################
make_panel_w_regression(ax, 1, 0, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_t2030, ssp4_t2030, ssp3_t2030, ssp2_t2030, ssp1_t2030, 
           "EIS", "Temperature ($^{\circ}$C)")

make_panel_w_regression(ax, 1, 1, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_t2060, ssp4_t2060, ssp3_t2060, ssp2_t2060, ssp1_t2060, 
           "EIS", None)

make_panel_w_regression(ax, 1, 2, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_t2100, ssp4_t2100, ssp3_t2100, ssp2_t2100, ssp1_t2100, 
           "EIS", None)

make_panel_w_regression(ax, 1, 3, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_t2150, ssp4_t2150, ssp3_t2150, ssp2_t2150, ssp1_t2150, 
           "EIS", None)

make_panel_w_regression(ax, 1, 4, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_t2200, ssp4_t2200, ssp3_t2200, ssp2_t2200, ssp1_t2200, 
           "EIS", None)

###########################
# PRTP ROW
###########################
make_panel_w_regression(ax, 2, 0, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_t2030, ssp4_t2030, ssp3_t2030, ssp2_t2030, ssp1_t2030, 
           r"$\delta$ (%)", "Temperature ($^{\circ}$C)")

make_panel_w_regression(ax, 2, 1, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_t2060, ssp4_t2060, ssp3_t2060, ssp2_t2060, ssp1_t2060, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 2, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_t2100, ssp4_t2100, ssp3_t2100, ssp2_t2100, ssp1_t2100, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 3, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_t2150, ssp4_t2150, ssp3_t2150, ssp2_t2150, ssp1_t2150, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 4, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_t2200, ssp4_t2200, ssp3_t2200, ssp2_t2200, ssp1_t2200, 
           r"$\delta$ (%)", None)

###########################
# EX TECH GROWTH ROW
###########################
make_panel_w_regression(ax, 3, 0, ssp5_tottech[:, 1], ssp4_tottech[:, 1], ssp3_tottech[:, 1], 
                        ssp2_tottech[:, 1], ssp1_tottech[:, 1], ssp5_t2030, ssp4_t2030, ssp3_t2030, 
                        ssp2_t2030, ssp1_t2030, "Tech Growth (%)", "Temperature ($^{\circ}$C)")

make_panel_w_regression(ax, 3, 1, ssp5_tottech[:, 2], ssp4_tottech[:, 2], ssp3_tottech[:, 2], 
                        ssp2_tottech[:, 2], ssp1_tottech[:, 2], ssp5_t2060, ssp4_t2060, ssp3_t2060, 
                        ssp2_t2060, ssp1_t2060, "Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 2, ssp5_tottech[:, 3], ssp4_tottech[:, 3], ssp3_tottech[:, 3], 
                        ssp2_tottech[:, 3], ssp1_tottech[:, 3], ssp5_t2100, ssp4_t2100, ssp3_t2100, 
                        ssp2_t2100, ssp1_t2100, "Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 3, ssp5_tottech[:, 4], ssp4_tottech[:, 4], ssp3_tottech[:, 4], 
                        ssp2_tottech[:, 4], ssp1_tottech[:, 4], ssp5_t2150, ssp4_t2150, ssp3_t2150, 
                        ssp2_t2150, ssp1_t2150, "Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 4, ssp5_tottech[:, 5], ssp4_tottech[:, 5], ssp3_tottech[:, 5], 
                        ssp2_tottech[:, 5], ssp1_tottech[:, 5], ssp5_t2200, ssp4_t2200, ssp3_t2200, 
                        ssp2_t2200, ssp1_t2200, "Tech Growth (%)", None)

ax[3,0].set_xticks([0,1,2,3])
ax[3,0].set_xlim((0,3))
ax[3,1].set_xticks([0,1,2,3,4,5])
ax[3,2].set_xticks([0,1,2,3,4,5,6])
ax[3,3].set_xticks([0,1,2,3,4,5,6])
ax[3,4].set_xticks([0,1,2,3,4,5,6])

fig.tight_layout()
sns.despine(trim=True, offset=10)

if save_fig:
    fig.savefig(''.join([basefile, 'regression-sticky-t.png']), dpi=400)    

In [ ]:
def get_tech_r2_in_time_alllin(tottech, paths):
    c2020, c2030, c2060, c2100, c2150, c2200 = paths.T
    _, _, r2_2020 = get_regression(tottech[:,0], c2020)
    _, _, r2_2030 = get_regression(tottech[:,1], c2030)
    _, _, r2_2060 = get_regression(tottech[:,2], c2060)
    _, _, r2_2100 = get_regression(tottech[:,3], c2100)
    _, _, r2_2150 = get_regression(tottech[:,4], c2150)
    _, _, r2_2200 = get_regression(tottech[:,5], c2200)
    
    r2 = np.array([r2_2020, r2_2030, r2_2060, r2_2100, r2_2150, r2_2200])
    return r2

In [ ]:
ssp5_eis_r2_t = get_r2_in_time(ssp5_eis, ssp5_exp_t)
ssp5_ra_r2_t = get_r2_in_time(ssp5_ra, ssp5_exp_t)
ssp5_prtp_r2_t = get_r2_in_time(ssp5_prtp, ssp5_exp_t)
ssp5_tech_r2_t = get_tech_r2_in_time_alllin(ssp5_tottech, ssp5_exp_t)

ssp4_eis_r2_t = get_r2_in_time(ssp4_eis, ssp4_exp_t)
ssp4_ra_r2_t = get_r2_in_time(ssp4_ra, ssp4_exp_t)
ssp4_prtp_r2_t = get_r2_in_time(ssp4_prtp, ssp4_exp_t)
ssp4_tech_r2_t = get_tech_r2_in_time_alllin(ssp4_tottech, ssp4_exp_t)

ssp3_eis_r2_t = get_r2_in_time(ssp3_eis, ssp3_exp_t)
ssp3_ra_r2_t = get_r2_in_time(ssp3_ra, ssp3_exp_t)
ssp3_prtp_r2_t = get_r2_in_time(ssp3_prtp, ssp3_exp_t)
ssp3_tech_r2_t = get_tech_r2_in_time_alllin(ssp3_tottech, ssp3_exp_t)

ssp2_eis_r2_t = get_r2_in_time(ssp2_eis, ssp2_exp_t)
ssp2_ra_r2_t = get_r2_in_time(ssp2_ra, ssp2_exp_t)
ssp2_prtp_r2_t = get_r2_in_time(ssp2_prtp, ssp2_exp_t)
ssp2_tech_r2_t = get_tech_r2_in_time_alllin(ssp2_tottech, ssp2_exp_t)

ssp1_eis_r2_t = get_r2_in_time(ssp1_eis, ssp1_exp_t)
ssp1_ra_r2_t = get_r2_in_time(ssp1_ra, ssp1_exp_t)
ssp1_prtp_r2_t = get_r2_in_time(ssp1_prtp, ssp1_exp_t)
ssp1_tech_r2_t = get_tech_r2_in_time_alllin(ssp1_tottech, ssp1_exp_t)

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(20,20), sharey=True)

ax[0].set_ylim((0,1))

alpha = 0.8
lw = 3
l1cind = 0
l2cind = 1
l3cind = 2
l4cind = 3
l5cind = 4

################################
# SSP5
################################
ax[0].plot(node_times[1:], ssp5_ra_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[0].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_t[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[0].plot(node_times[1:], ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[0].fill_between(node_times[1:], ssp5_ra_r2_t[1:], ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[0].plot(node_times[1:], ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:] + ssp5_prtp_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[0].fill_between(node_times[1:], ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:], ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:] + ssp5_prtp_r2_t[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[0].plot(node_times[1:], ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:] + ssp5_prtp_r2_t[1:] + ssp5_tech_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[0].fill_between(node_times[1:], ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:] + ssp5_prtp_r2_t[1:], ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:] + ssp5_prtp_r2_t[1:] + ssp5_tech_r2_t[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
ax[1].plot(node_times[1:], ssp4_ra_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[1].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp4_ra_r2_t[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[1].plot(node_times[1:], ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[1].fill_between(node_times[1:], ssp4_ra_r2_t[1:], ssp5_ra_r2_t[1:] + ssp4_eis_r2_t[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[1].plot(node_times[1:], ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:] + ssp4_prtp_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[1].fill_between(node_times[1:], ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:], ssp4_ra_r2_t[1:] + ssp4_eis_r2_t[1:] + ssp4_prtp_r2_t[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[1].plot(node_times[1:], ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:] + ssp4_prtp_r2_t[1:] + ssp4_tech_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[1].fill_between(node_times[1:], ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:] + ssp4_prtp_r2_t[1:], ssp4_ra_r2_t[1:] + ssp4_eis_r2_t[1:] + ssp4_prtp_r2_t[1:] + ssp4_tech_r2_t[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ax[2].plot(node_times[1:], ssp3_ra_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[2].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp3_ra_r2_t[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[2].plot(node_times[1:], ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[2].fill_between(node_times[1:], ssp3_ra_r2_t[1:], ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[2].plot(node_times[1:], ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:] + ssp3_prtp_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[2].fill_between(node_times[1:], ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:], ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:] + ssp3_prtp_r2_t[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[2].plot(node_times[1:], ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:] + ssp3_prtp_r2_t[1:] + ssp3_tech_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[2].fill_between(node_times[1:], ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:] + ssp3_prtp_r2_t[1:], ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:] + ssp3_prtp_r2_t[1:] + ssp3_tech_r2_t[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ax[3].plot(node_times[1:], ssp2_ra_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[3].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp2_ra_r2_t[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[3].plot(node_times[1:], ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[3].fill_between(node_times[1:], ssp2_ra_r2_t[1:], ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[3].plot(node_times[1:], ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:] + ssp2_prtp_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[3].fill_between(node_times[1:], ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:], ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:] + ssp2_prtp_r2_t[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[3].plot(node_times[1:], ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:] + ssp2_prtp_r2_t[1:] + ssp2_tech_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[3].fill_between(node_times[1:], ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:] + ssp2_prtp_r2_t[1:], ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:] + ssp2_prtp_r2_t[1:] + ssp2_tech_r2_t[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ax[4].plot(node_times[1:], ssp1_ra_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[4].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_t[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[4].plot(node_times[1:], ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[4].fill_between(node_times[1:], ssp1_ra_r2_t[1:], ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[4].plot(node_times[1:], ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:] + ssp1_prtp_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[4].fill_between(node_times[1:], ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:], ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:] + ssp1_prtp_r2_t[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[4].plot(node_times[1:], ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:] + ssp1_prtp_r2_t[1:] + ssp1_tech_r2_t[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[4].fill_between(node_times[1:], ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:] + ssp1_prtp_r2_t[1:], ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:] + ssp1_prtp_r2_t[1:] + ssp1_tech_r2_t[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

ax[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=5)

ax[0].set_ylabel("r$^{2}$")
ax[0].set_title("SSP5")
ax[1].set_title("SSP4")
ax[2].set_title("SSP3")
ax[3].set_title("SSP2")
ax[4].set_title("SSP1")

for i in range(5):
    ax[i].set_xlim((node_times[1], node_times[-1]))
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_xlabel("Year")

#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(20,20), sharey=True)

ax[0].set_ylim((0,1))

alpha = 0.8
lw = 3
l1cind = 0
l2cind = 1
l3cind = 2
l4cind = 3
l5cind = 4

################################
# SSP5
################################
tot_ssp5 = ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:] + ssp5_prtp_r2_t[1:] + ssp5_tech_r2_t[1:]
ax[0].plot(node_times[1:], ssp5_ra_r2_t[1:] * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[0].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_t[1:] * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[0].plot(node_times[1:], (ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[0].fill_between(node_times[1:], ssp5_ra_r2_t[1:]  * tot_ssp5**(-1), 
                   (ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[0].plot(node_times[1:], (ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:] + ssp5_prtp_r2_t[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[0].fill_between(node_times[1:], (ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:]) * tot_ssp5**(-1), 
                  (ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:] + ssp5_prtp_r2_t[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[0].plot(node_times[1:], (ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:] + ssp5_prtp_r2_t[1:] + ssp5_tech_r2_t[1:]) * tot_ssp5**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax[0].fill_between(node_times[1:], (ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:] + ssp5_prtp_r2_t[1:]) * tot_ssp5**(-1), 
                   (ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:] + ssp5_prtp_r2_t[1:] + ssp5_tech_r2_t[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
tot_ssp4 = ssp4_ra_r2_t[1:] + ssp4_eis_r2_t[1:] + ssp4_prtp_r2_t[1:] + ssp4_tech_r2_t[1:]
ax[1].plot(node_times[1:], ssp4_ra_r2_t[1:] * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[1].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp4_ra_r2_t[1:] * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[1].plot(node_times[1:], (ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[1].fill_between(node_times[1:], ssp4_ra_r2_t[1:] * tot_ssp4**(-1), 
                   (ssp4_ra_r2_t[1:] + ssp4_eis_r2_t[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[1].plot(node_times[1:], (ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:] + ssp4_prtp_r2_t[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[1].fill_between(node_times[1:], (ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:]) * tot_ssp4**(-1), 
                  (ssp4_ra_r2_t[1:] + ssp4_eis_r2_t[1:] + ssp4_prtp_r2_t[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[1].plot(node_times[1:], (ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:] + ssp4_prtp_r2_t[1:] + ssp4_tech_r2_t[1:]) * tot_ssp4**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax[1].fill_between(node_times[1:], (ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:] + ssp4_prtp_r2_t[1:]) * tot_ssp4**(-1), 
                   (ssp4_ra_r2_t[1:] + ssp4_eis_r2_t[1:] + ssp4_prtp_r2_t[1:] + ssp4_tech_r2_t[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ssp3_tot = ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:] + ssp3_prtp_r2_t[1:] + ssp3_tech_r2_t[1:]
ax[2].plot(node_times[1:], ssp3_ra_r2_t[1:] * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[2].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp3_ra_r2_t[1:] * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[2].plot(node_times[1:], (ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[2].fill_between(node_times[1:], ssp3_ra_r2_t[1:] * ssp3_tot**(-1), 
                   (ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[2].plot(node_times[1:], (ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:] + ssp3_prtp_r2_t[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[2].fill_between(node_times[1:], (ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:] + ssp3_prtp_r2_t[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[2].plot(node_times[1:], (ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:] + ssp3_prtp_r2_t[1:] + ssp3_tech_r2_t[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[2].fill_between(node_times[1:], (ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:] + ssp3_prtp_r2_t[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:] + ssp3_prtp_r2_t[1:] + ssp3_tech_r2_t[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ssp2_tot = ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:] + ssp2_prtp_r2_t[1:] + ssp2_tech_r2_t[1:]
ax[3].plot(node_times[1:], ssp2_ra_r2_t[1:] * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[3].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp2_ra_r2_t[1:]* ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[3].plot(node_times[1:], (ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:])* ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[3].fill_between(node_times[1:], ssp2_ra_r2_t[1:] * ssp2_tot**(-1), 
                   (ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[3].plot(node_times[1:], (ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:] + ssp2_prtp_r2_t[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[3].fill_between(node_times[1:], (ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:] + ssp2_prtp_r2_t[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[3].plot(node_times[1:], (ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:] + ssp2_prtp_r2_t[1:] + ssp2_tech_r2_t[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[3].fill_between(node_times[1:], (ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:] + ssp2_prtp_r2_t[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:] + ssp2_prtp_r2_t[1:] + ssp2_tech_r2_t[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ssp1_tot = ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:] + ssp1_prtp_r2_t[1:] + ssp1_tech_r2_t[1:]
ax[4].plot(node_times[1:], ssp1_ra_r2_t[1:] * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[4].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_t[1:] * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[4].plot(node_times[1:], (ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[4].fill_between(node_times[1:], ssp1_ra_r2_t[1:] * ssp1_tot**(-1), 
                   (ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l2cind], label="EIS")

ax[4].plot(node_times[1:], (ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:] + ssp1_prtp_r2_t[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[4].fill_between(node_times[1:], (ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:] + ssp1_prtp_r2_t[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l3cind], label="PRTP")

ax[4].plot(node_times[1:], (ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:] + ssp1_prtp_r2_t[1:] + ssp1_tech_r2_t[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[4].fill_between(node_times[1:], (ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:] + ssp1_prtp_r2_t[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:] + ssp1_prtp_r2_t[1:] + ssp1_tech_r2_t[1:]) * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

ax[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=5, fontsize=20)

ax[0].set_ylabel("$r^{2} / r_{tot}^{2}$")
ax[0].set_title("SSP5")
ax[1].set_title("SSP4")
ax[2].set_title("SSP3")
ax[3].set_title("SSP2")
ax[4].set_title("SSP1")

panel_label = ["A", "B", "C", "D", "E"]
for i in range(5):
    ax[i].set_xlim((node_times[1], node_times[-1]))
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_xlabel("Year")
    ax[i].text(2168, 0.88, panel_label[i], fontsize=22, color='white', fontweight='bold')
    ax[i].set_xticks([2030,2100,2200])
    ax[i].set_xticklabels(['2030', '2100', '2200'])
    ax[i].tick_params(axis='both', labelsize=14)

sns.despine(trim=True, offset=10)

if save_fig:
    fig.savefig(''.join([basefile, 'rsquared-t.png']), dpi=400)

# Concentrations! 

In [ ]:
fig, ax = plt.subplots(4, 5, figsize=(30, 20))

ax[0,0].set_title("2030")
ax[0,1].set_title("2060")
ax[0,2].set_title("2100")
ax[0,3].set_title("2150")
ax[0,4].set_title("2200")

###########################
## RISK AVERSION ROW
###########################
make_panel_w_regression(ax, 0, 0, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_co2030, ssp4_co2030, ssp3_co2030, ssp2_co2030, ssp1_co2030, 
           "Risk Aversion", "CO$_2$ conc. (ppm)")

make_panel_w_regression(ax, 0, 1, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_co2060, ssp4_co2060, ssp3_co2060, ssp2_co2060, ssp1_co2060, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 2, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_co2100, ssp4_co2100, ssp3_co2100, ssp2_co2100, ssp1_co2100, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 3, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_co2150, ssp4_co2150, ssp3_co2150, ssp2_co2150, ssp1_co2150, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 4, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_co2200, ssp4_co2200, ssp3_co2200, ssp2_co2200, ssp1_co2200, 
           "Risk Aversion", None)

###########################
# EIS ROW
###########################
make_panel_w_regression(ax, 1, 0, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_co2030, ssp4_co2030, ssp3_co2030, ssp2_co2030, ssp1_co2030, 
           "EIS", "CO$_2$ conc. (ppm)")

make_panel_w_regression(ax, 1, 1, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_co2060, ssp4_co2060, ssp3_co2060, ssp2_co2060, ssp1_co2060, 
           "EIS", None)

make_panel_w_regression(ax, 1, 2, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_co2100, ssp4_co2100, ssp3_co2100, ssp2_co2100, ssp1_co2100, 
           "EIS", None)

make_panel_w_regression(ax, 1, 3, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_co2150, ssp4_co2150, ssp3_co2150, ssp2_co2150, ssp1_co2150, 
           "EIS", None)

make_panel_w_regression(ax, 1, 4, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_co2200, ssp4_co2200, ssp3_co2200, ssp2_co2200, ssp1_co2200, 
           "EIS", None)

###########################
# PRTP ROW
###########################
make_panel_w_regression(ax, 2, 0, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_co2030, ssp4_co2030, ssp3_co2030, ssp2_co2030, ssp1_co2030, 
           r"$\delta$ (%)", "CO$_2$ conc. (ppm)")

make_panel_w_regression(ax, 2, 1, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_co2060, ssp4_co2060, ssp3_co2060, ssp2_co2060, ssp1_co2060, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 2, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_co2100, ssp4_co2100, ssp3_co2100, ssp2_co2100, ssp1_co2100, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 3, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_co2150, ssp4_co2150, ssp3_co2150, ssp2_co2150, ssp1_co2150, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 4, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_co2200, ssp4_co2200, ssp3_co2200, ssp2_co2200, ssp1_co2200, 
           r"$\delta$ (%)", None)

###########################
# EX TECH GROWTH ROW
###########################
make_panel_w_regression(ax, 3, 0, ssp5_tottech[:, 1], ssp4_tottech[:, 1], ssp3_tottech[:, 1], 
                        ssp2_tottech[:, 1], ssp1_tottech[:, 1], ssp5_co2030, ssp4_co2030, ssp3_co2030, 
                        ssp2_co2030, ssp1_co2030, "Tech Growth (%)", "CO$_2$ conc. (ppm)")

make_panel_w_regression(ax, 3, 1, ssp5_tottech[:, 2], ssp4_tottech[:, 2], ssp3_tottech[:, 2], 
                        ssp2_tottech[:, 2], ssp1_tottech[:, 2], ssp5_co2060, ssp4_co2060, ssp3_co2060, 
                        ssp2_co2060, ssp1_co2060, "Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 2, ssp5_tottech[:, 3], ssp4_tottech[:, 3], ssp3_tottech[:, 3], 
                        ssp2_tottech[:, 3], ssp1_tottech[:, 3], ssp5_co2100, ssp4_co2100, ssp3_co2100, 
                        ssp2_co2100, ssp1_co2100, "Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 3, ssp5_tottech[:, 4], ssp4_tottech[:, 4], ssp3_tottech[:, 4], 
                        ssp2_tottech[:, 4], ssp1_tottech[:, 4], ssp5_co2150, ssp4_co2150, ssp3_co2150, 
                        ssp2_co2150, ssp1_co2150, "Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 4, ssp5_tottech[:, 5], ssp4_tottech[:, 5], ssp3_tottech[:, 5], 
                        ssp2_tottech[:, 5], ssp1_tottech[:, 5], ssp5_co2200, ssp4_co2200, ssp3_co2200, 
                        ssp2_co2200, ssp1_co2200, "Tech Growth (%)", None)

ax[3,0].set_xticks([0,1,2,3])
ax[3,0].set_xlim((0,3))
ax[3,1].set_xticks([0,1,2,3,4,5])
ax[3,2].set_xticks([0,1,2,3,4,5,6])
ax[3,3].set_xticks([0,1,2,3,4,5,6])
ax[3,4].set_xticks([0,1,2,3,4,5,6])

fig.tight_layout()
sns.despine(trim=True, offset=10)

if save_fig:
    fig.savefig(''.join([basefile, 'regression-sticky-co.png']), dpi=400)    

In [ ]:
ssp5_eis_r2_co = get_r2_in_time(ssp5_eis, ssp5_exp_co2)
ssp5_ra_r2_co = get_r2_in_time(ssp5_ra, ssp5_exp_co2)
ssp5_prtp_r2_co = get_r2_in_time(ssp5_prtp, ssp5_exp_co2)
ssp5_tech_r2_co = get_tech_r2_in_time_alllin(ssp5_tottech, ssp5_exp_co2)

ssp4_eis_r2_co = get_r2_in_time(ssp4_eis, ssp4_exp_co2)
ssp4_ra_r2_co = get_r2_in_time(ssp4_ra, ssp4_exp_co2)
ssp4_prtp_r2_co = get_r2_in_time(ssp4_prtp, ssp4_exp_co2)
ssp4_tech_r2_co = get_tech_r2_in_time_alllin(ssp4_tottech, ssp4_exp_co2)

ssp3_eis_r2_co = get_r2_in_time(ssp3_eis, ssp3_exp_co2)
ssp3_ra_r2_co = get_r2_in_time(ssp3_ra, ssp3_exp_co2)
ssp3_prtp_r2_co = get_r2_in_time(ssp3_prtp, ssp3_exp_co2)
ssp3_tech_r2_co = get_tech_r2_in_time_alllin(ssp3_tottech, ssp3_exp_co2)

ssp2_eis_r2_co = get_r2_in_time(ssp2_eis, ssp2_exp_co2)
ssp2_ra_r2_co = get_r2_in_time(ssp2_ra, ssp2_exp_co2)
ssp2_prtp_r2_co = get_r2_in_time(ssp2_prtp, ssp2_exp_co2)
ssp2_tech_r2_co = get_tech_r2_in_time_alllin(ssp2_tottech, ssp2_exp_co2)

ssp1_eis_r2_co = get_r2_in_time(ssp1_eis, ssp1_exp_co2)
ssp1_ra_r2_co = get_r2_in_time(ssp1_ra, ssp1_exp_co2)
ssp1_prtp_r2_co = get_r2_in_time(ssp1_prtp, ssp1_exp_co2)
ssp1_tech_r2_co = get_tech_r2_in_time_alllin(ssp1_tottech, ssp1_exp_co2)

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(20,20), sharey=True)

ax[0].set_ylim((0,1))

alpha = 0.8
lw = 3
l1cind = 0
l2cind = 1
l3cind = 2
l4cind = 3
l5cind = 4

################################
# SSP5
################################
ax[0].plot(node_times[1:], ssp5_ra_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[0].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_co[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[0].plot(node_times[1:], ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[0].fill_between(node_times[1:], ssp5_ra_r2_co[1:], ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[0].plot(node_times[1:], ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:] + ssp5_prtp_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[0].fill_between(node_times[1:], ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:], ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:] + ssp5_prtp_r2_co[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[0].plot(node_times[1:], ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:] + ssp5_prtp_r2_co[1:] + ssp5_tech_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[0].fill_between(node_times[1:], ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:] + ssp5_prtp_r2_co[1:], ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:] + ssp5_prtp_r2_co[1:] + ssp5_tech_r2_co[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
ax[1].plot(node_times[1:], ssp4_ra_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[1].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp4_ra_r2_co[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[1].plot(node_times[1:], ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[1].fill_between(node_times[1:], ssp4_ra_r2_co[1:], ssp5_ra_r2_co[1:] + ssp4_eis_r2_co[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[1].plot(node_times[1:], ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:] + ssp4_prtp_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[1].fill_between(node_times[1:], ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:], ssp4_ra_r2_co[1:] + ssp4_eis_r2_co[1:] + ssp4_prtp_r2_co[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[1].plot(node_times[1:], ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:] + ssp4_prtp_r2_co[1:] + ssp4_tech_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[1].fill_between(node_times[1:], ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:] + ssp4_prtp_r2_co[1:], ssp4_ra_r2_co[1:] + ssp4_eis_r2_co[1:] + ssp4_prtp_r2_co[1:] + ssp4_tech_r2_co[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ax[2].plot(node_times[1:], ssp3_ra_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[2].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp3_ra_r2_co[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[2].plot(node_times[1:], ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[2].fill_between(node_times[1:], ssp3_ra_r2_co[1:], ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[2].plot(node_times[1:], ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:] + ssp3_prtp_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[2].fill_between(node_times[1:], ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:], ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:] + ssp3_prtp_r2_co[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[2].plot(node_times[1:], ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:] + ssp3_prtp_r2_co[1:] + ssp3_tech_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[2].fill_between(node_times[1:], ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:] + ssp3_prtp_r2_co[1:], ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:] + ssp3_prtp_r2_co[1:] + ssp3_tech_r2_co[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ax[3].plot(node_times[1:], ssp2_ra_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[3].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp2_ra_r2_co[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[3].plot(node_times[1:], ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[3].fill_between(node_times[1:], ssp2_ra_r2_co[1:], ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[3].plot(node_times[1:], ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:] + ssp2_prtp_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[3].fill_between(node_times[1:], ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:], ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:] + ssp2_prtp_r2_co[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[3].plot(node_times[1:], ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:] + ssp2_prtp_r2_co[1:] + ssp2_tech_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[3].fill_between(node_times[1:], ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:] + ssp2_prtp_r2_co[1:], ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:] + ssp2_prtp_r2_co[1:] + ssp2_tech_r2_co[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ax[4].plot(node_times[1:], ssp1_ra_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[4].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_co[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[4].plot(node_times[1:], ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[4].fill_between(node_times[1:], ssp1_ra_r2_co[1:], ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[4].plot(node_times[1:], ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:] + ssp1_prtp_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[4].fill_between(node_times[1:], ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:], ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:] + ssp1_prtp_r2_co[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[4].plot(node_times[1:], ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:] + ssp1_prtp_r2_co[1:] + ssp1_tech_r2_co[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[4].fill_between(node_times[1:], ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:] + ssp1_prtp_r2_co[1:], ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:] + ssp1_prtp_r2_co[1:] + ssp1_tech_r2_co[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

ax[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=5)

ax[0].set_ylabel("r$^{2}$")
ax[0].set_title("SSP5")
ax[1].set_title("SSP4")
ax[2].set_title("SSP3")
ax[3].set_title("SSP2")
ax[4].set_title("SSP1")

for i in range(5):
    ax[i].set_xlim((node_times[1], node_times[-1]))
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_xlabel("Year")

#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(20,20), sharey=True)

ax[0].set_ylim((0,1))

alpha = 0.8
lw = 3
l1cind = 0
l2cind = 1
l3cind = 2
l4cind = 3
l5cind = 4

################################
# SSP5
################################
tot_ssp5 = ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:] + ssp5_prtp_r2_co[1:] + ssp5_tech_r2_co[1:]
ax[0].plot(node_times[1:], ssp5_ra_r2_co[1:] * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[0].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_co[1:] * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[0].plot(node_times[1:], (ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[0].fill_between(node_times[1:], ssp5_ra_r2_co[1:]  * tot_ssp5**(-1), 
                   (ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[0].plot(node_times[1:], (ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:] + ssp5_prtp_r2_co[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[0].fill_between(node_times[1:], (ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:]) * tot_ssp5**(-1), 
                  (ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:] + ssp5_prtp_r2_co[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[0].plot(node_times[1:], (ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:] + ssp5_prtp_r2_co[1:] + ssp5_tech_r2_co[1:]) * tot_ssp5**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax[0].fill_between(node_times[1:], (ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:] + ssp5_prtp_r2_co[1:]) * tot_ssp5**(-1), 
                   (ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:] + ssp5_prtp_r2_co[1:] + ssp5_tech_r2_co[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
tot_ssp4 = ssp4_ra_r2_co[1:] + ssp4_eis_r2_co[1:] + ssp4_prtp_r2_co[1:] + ssp4_tech_r2_co[1:]
ax[1].plot(node_times[1:], ssp4_ra_r2_co[1:] * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[1].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp4_ra_r2_co[1:] * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[1].plot(node_times[1:], (ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[1].fill_between(node_times[1:], ssp4_ra_r2_co[1:] * tot_ssp4**(-1), 
                   (ssp4_ra_r2_co[1:] + ssp4_eis_r2_co[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[1].plot(node_times[1:], (ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:] + ssp4_prtp_r2_co[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[1].fill_between(node_times[1:], (ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:]) * tot_ssp4**(-1), 
                  (ssp4_ra_r2_co[1:] + ssp4_eis_r2_co[1:] + ssp4_prtp_r2_co[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[1].plot(node_times[1:], (ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:] + ssp4_prtp_r2_co[1:] + ssp4_tech_r2_co[1:]) * tot_ssp4**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax[1].fill_between(node_times[1:], (ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:] + ssp4_prtp_r2_co[1:]) * tot_ssp4**(-1), 
                   (ssp4_ra_r2_co[1:] + ssp4_eis_r2_co[1:] + ssp4_prtp_r2_co[1:] + ssp4_tech_r2_co[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ssp3_tot = ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:] + ssp3_prtp_r2_co[1:] + ssp3_tech_r2_co[1:]
ax[2].plot(node_times[1:], ssp3_ra_r2_co[1:] * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[2].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp3_ra_r2_co[1:] * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[2].plot(node_times[1:], (ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[2].fill_between(node_times[1:], ssp3_ra_r2_co[1:] * ssp3_tot**(-1), 
                   (ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[2].plot(node_times[1:], (ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:] + ssp3_prtp_r2_co[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[2].fill_between(node_times[1:], (ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:] + ssp3_prtp_r2_co[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[2].plot(node_times[1:], (ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:] + ssp3_prtp_r2_co[1:] + ssp3_tech_r2_co[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[2].fill_between(node_times[1:], (ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:] + ssp3_prtp_r2_co[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:] + ssp3_prtp_r2_co[1:] + ssp3_tech_r2_co[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ssp2_tot = ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:] + ssp2_prtp_r2_co[1:] + ssp2_tech_r2_co[1:]
ax[3].plot(node_times[1:], ssp2_ra_r2_co[1:] * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[3].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp2_ra_r2_co[1:]* ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[3].plot(node_times[1:], (ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:])* ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[3].fill_between(node_times[1:], ssp2_ra_r2_co[1:] * ssp2_tot**(-1), 
                   (ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[3].plot(node_times[1:], (ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:] + ssp2_prtp_r2_co[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[3].fill_between(node_times[1:], (ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:] + ssp2_prtp_r2_co[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[3].plot(node_times[1:], (ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:] + ssp2_prtp_r2_co[1:] + ssp2_tech_r2_co[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[3].fill_between(node_times[1:], (ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:] + ssp2_prtp_r2_co[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:] + ssp2_prtp_r2_co[1:] + ssp2_tech_r2_co[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ssp1_tot = ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:] + ssp1_prtp_r2_co[1:] + ssp1_tech_r2_co[1:]
ax[4].plot(node_times[1:], ssp1_ra_r2_co[1:] * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[4].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_co[1:] * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[4].plot(node_times[1:], (ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[4].fill_between(node_times[1:], ssp1_ra_r2_co[1:] * ssp1_tot**(-1), 
                   (ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l2cind], label="EIS")

ax[4].plot(node_times[1:], (ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:] + ssp1_prtp_r2_co[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[4].fill_between(node_times[1:], (ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:] + ssp1_prtp_r2_co[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l3cind], label="PRTP")

ax[4].plot(node_times[1:], (ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:] + ssp1_prtp_r2_co[1:] + ssp1_tech_r2_co[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[4].fill_between(node_times[1:], (ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:] + ssp1_prtp_r2_co[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:] + ssp1_prtp_r2_co[1:] + ssp1_tech_r2_co[1:]) * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

ax[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=5, fontsize=20)

ax[0].set_ylabel("$r^{2} / r_{tot}^{2}$")
ax[0].set_title("SSP5")
ax[1].set_title("SSP4")
ax[2].set_title("SSP3")
ax[3].set_title("SSP2")
ax[4].set_title("SSP1")

panel_label = ["A", "B", "C", "D", "E"]
for i in range(5):
    ax[i].set_xlim((node_times[1], node_times[-1]))
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_xlabel("Year")
    ax[i].text(2168, 0.88, panel_label[i], fontsize=22, color='white', fontweight='bold')
    ax[i].set_xticks([2030,2100,2200])
    ax[i].set_xticklabels(['2030', '2100', '2200'])
    ax[i].tick_params(axis='both', labelsize=14)

sns.despine(trim=True, offset=10)

if save_fig:
    fig.savefig(''.join([basefile, 'rsquared-conc.png']), dpi=400)

# Damages!!

In [ ]:
fig, ax = plt.subplots(4, 5, figsize=(30, 20))

ax[0,0].set_title("2030")
ax[0,1].set_title("2060")
ax[0,2].set_title("2100")
ax[0,3].set_title("2150")
ax[0,4].set_title("2200")

###########################
## RISK AVERSION ROW
###########################
make_panel_w_regression(ax, 0, 0, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_d2030, ssp4_d2030, ssp3_d2030, ssp2_d2030, ssp1_d2030, 
           "Risk Aversion", "Damage (frac GDP)")

make_panel_w_regression(ax, 0, 1, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_d2060, ssp4_d2060, ssp3_d2060, ssp2_d2060, ssp1_d2060, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 2, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_d2100, ssp4_d2100, ssp3_d2100, ssp2_d2100, ssp1_d2100, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 3, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_d2150, ssp4_d2150, ssp3_d2150, ssp2_d2150, ssp1_d2150, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 4, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_d2200, ssp4_d2200, ssp3_d2200, ssp2_d2200, ssp1_d2200, 
           "Risk Aversion", None)

###########################
# EIS ROW
###########################
make_panel_w_regression(ax, 1, 0, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_d2030, ssp4_d2030, ssp3_d2030, ssp2_d2030, ssp1_d2030, 
           "EIS", "Damage (frac GDP)")

make_panel_w_regression(ax, 1, 1, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_d2060, ssp4_d2060, ssp3_d2060, ssp2_d2060, ssp1_d2060, 
           "EIS", None)

make_panel_w_regression(ax, 1, 2, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_d2100, ssp4_d2100, ssp3_d2100, ssp2_d2100, ssp1_d2100, 
           "EIS", None)

make_panel_w_regression(ax, 1, 3, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_d2150, ssp4_d2150, ssp3_d2150, ssp2_d2150, ssp1_d2150, 
           "EIS", None)

make_panel_w_regression(ax, 1, 4, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_d2200, ssp4_d2200, ssp3_d2200, ssp2_d2200, ssp1_d2200, 
           "EIS", None)

###########################
# PRTP ROW
###########################
make_panel_w_regression(ax, 2, 0, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_d2030, ssp4_d2030, ssp3_d2030, ssp2_d2030, ssp1_d2030, 
           r"$\delta$ (%)", "Damage (frac GDP)")

make_panel_w_regression(ax, 2, 1, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_d2060, ssp4_d2060, ssp3_d2060, ssp2_d2060, ssp1_d2060, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 2, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_d2100, ssp4_d2100, ssp3_d2100, ssp2_d2100, ssp1_d2100, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 3, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_d2150, ssp4_d2150, ssp3_d2150, ssp2_d2150, ssp1_d2150, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 4, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_d2200, ssp4_d2200, ssp3_d2200, ssp2_d2200, ssp1_d2200, 
           r"$\delta$ (%)", None)

###########################
# EX TECH GROWTH ROW
###########################
make_panel_w_regression(ax, 3, 0, ssp5_tottech[:, 1], ssp4_tottech[:, 1], ssp3_tottech[:, 1], 
                        ssp2_tottech[:, 1], ssp1_tottech[:, 1], ssp5_d2030, ssp4_d2030, ssp3_d2030, 
                        ssp2_d2030, ssp1_d2030, "Tech Growth (%)", "Damage (frac GDP)")

make_panel_w_regression(ax, 3, 1, ssp5_tottech[:, 2], ssp4_tottech[:, 2], ssp3_tottech[:, 2], 
                        ssp2_tottech[:, 2], ssp1_tottech[:, 2], ssp5_d2060, ssp4_d2060, ssp3_d2060, 
                        ssp2_d2060, ssp1_d2060, "Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 2, ssp5_tottech[:, 3], ssp4_tottech[:, 3], ssp3_tottech[:, 3], 
                        ssp2_tottech[:, 3], ssp1_tottech[:, 3], ssp5_d2100, ssp4_d2100, ssp3_d2100, 
                        ssp2_d2100, ssp1_d2100, "Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 3, ssp5_tottech[:, 4], ssp4_tottech[:, 4], ssp3_tottech[:, 4], 
                        ssp2_tottech[:, 4], ssp1_tottech[:, 4], ssp5_d2150, ssp4_d2150, ssp3_d2150, 
                        ssp2_d2150, ssp1_d2150, "Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 4, ssp5_tottech[:, 5], ssp4_tottech[:, 5], ssp3_tottech[:, 5], 
                        ssp2_tottech[:, 5], ssp1_tottech[:, 5], ssp5_d2200, ssp4_d2200, ssp3_d2200, 
                        ssp2_d2200, ssp1_d2200, "Tech Growth (%)", None)

ax[3,0].set_xticks([0,1,2,3])
ax[3,0].set_xlim((0,3))
ax[3,1].set_xticks([0,1,2,3,4,5])
ax[3,2].set_xticks([0,1,2,3,4,5,6])
ax[3,3].set_xticks([0,1,2,3,4,5,6])
ax[3,4].set_xticks([0,1,2,3,4,5,6])

fig.tight_layout()
sns.despine(trim=True, offset=10)

if save_fig:
    fig.savefig(''.join([basefile, 'regression-sticky-dam.png']), dpi=400)    

In [ ]:
ssp5_eis_r2_d = get_r2_in_time(ssp5_eis, ssp5_exp_dam)
ssp5_ra_r2_d = get_r2_in_time(ssp5_ra, ssp5_exp_dam)
ssp5_prtp_r2_d = get_r2_in_time(ssp5_prtp, ssp5_exp_dam)
ssp5_tech_r2_d = get_tech_r2_in_time_alllin(ssp5_tottech, ssp5_exp_dam)

ssp4_eis_r2_d = get_r2_in_time(ssp4_eis, ssp4_exp_dam)
ssp4_ra_r2_d = get_r2_in_time(ssp4_ra, ssp4_exp_dam)
ssp4_prtp_r2_d = get_r2_in_time(ssp4_prtp, ssp4_exp_dam)
ssp4_tech_r2_d = get_tech_r2_in_time_alllin(ssp4_tottech, ssp4_exp_dam)

ssp3_eis_r2_d = get_r2_in_time(ssp3_eis, ssp3_exp_dam)
ssp3_ra_r2_d = get_r2_in_time(ssp3_ra, ssp3_exp_dam)
ssp3_prtp_r2_d = get_r2_in_time(ssp3_prtp, ssp3_exp_dam)
ssp3_tech_r2_d = get_tech_r2_in_time_alllin(ssp3_tottech, ssp3_exp_dam)

ssp2_eis_r2_d = get_r2_in_time(ssp2_eis, ssp2_exp_dam)
ssp2_ra_r2_d = get_r2_in_time(ssp2_ra, ssp2_exp_dam)
ssp2_prtp_r2_d = get_r2_in_time(ssp2_prtp, ssp2_exp_dam)
ssp2_tech_r2_d = get_tech_r2_in_time_alllin(ssp2_tottech, ssp2_exp_dam)

ssp1_eis_r2_d = get_r2_in_time(ssp1_eis, ssp1_exp_dam)
ssp1_ra_r2_d = get_r2_in_time(ssp1_ra, ssp1_exp_dam)
ssp1_prtp_r2_d = get_r2_in_time(ssp1_prtp, ssp1_exp_dam)
ssp1_tech_r2_d = get_tech_r2_in_time_alllin(ssp1_tottech, ssp1_exp_dam)

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(20,20), sharey=True)

ax[0].set_ylim((0,1))

alpha = 0.8
lw = 3
l1cind = 0
l2cind = 1
l3cind = 2
l4cind = 3
l5cind = 4

################################
# SSP5
################################
ax[0].plot(node_times[1:], ssp5_ra_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[0].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_d[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[0].plot(node_times[1:], ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[0].fill_between(node_times[1:], ssp5_ra_r2_d[1:], ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[0].plot(node_times[1:], ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:] + ssp5_prtp_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[0].fill_between(node_times[1:], ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:], ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:] + ssp5_prtp_r2_d[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[0].plot(node_times[1:], ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:] + ssp5_prtp_r2_d[1:] + ssp5_tech_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[0].fill_between(node_times[1:], ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:] + ssp5_prtp_r2_d[1:], ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:] + ssp5_prtp_r2_d[1:] + ssp5_tech_r2_d[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
ax[1].plot(node_times[1:], ssp4_ra_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[1].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp4_ra_r2_d[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[1].plot(node_times[1:], ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[1].fill_between(node_times[1:], ssp4_ra_r2_d[1:], ssp5_ra_r2_d[1:] + ssp4_eis_r2_d[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[1].plot(node_times[1:], ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:] + ssp4_prtp_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[1].fill_between(node_times[1:], ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:], ssp4_ra_r2_d[1:] + ssp4_eis_r2_d[1:] + ssp4_prtp_r2_d[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[1].plot(node_times[1:], ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:] + ssp4_prtp_r2_d[1:] + ssp4_tech_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[1].fill_between(node_times[1:], ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:] + ssp4_prtp_r2_d[1:], ssp4_ra_r2_d[1:] + ssp4_eis_r2_d[1:] + ssp4_prtp_r2_d[1:] + ssp4_tech_r2_d[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ax[2].plot(node_times[1:], ssp3_ra_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[2].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp3_ra_r2_d[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[2].plot(node_times[1:], ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[2].fill_between(node_times[1:], ssp3_ra_r2_d[1:], ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[2].plot(node_times[1:], ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:] + ssp3_prtp_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[2].fill_between(node_times[1:], ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:], ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:] + ssp3_prtp_r2_d[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[2].plot(node_times[1:], ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:] + ssp3_prtp_r2_d[1:] + ssp3_tech_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[2].fill_between(node_times[1:], ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:] + ssp3_prtp_r2_d[1:], ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:] + ssp3_prtp_r2_d[1:] + ssp3_tech_r2_d[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ax[3].plot(node_times[1:], ssp2_ra_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[3].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp2_ra_r2_d[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[3].plot(node_times[1:], ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[3].fill_between(node_times[1:], ssp2_ra_r2_d[1:], ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[3].plot(node_times[1:], ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:] + ssp2_prtp_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[3].fill_between(node_times[1:], ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:], ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:] + ssp2_prtp_r2_d[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[3].plot(node_times[1:], ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:], ssp2_prtp_r2_d[1:] + ssp2_tech_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[3].fill_between(node_times[1:], ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:] + ssp2_prtp_r2_d[1:], ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:] + ssp2_prtp_r2_d[1:] + ssp2_tech_r2_d[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ax[4].plot(node_times[1:], ssp1_ra_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[4].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_d[1:], alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[4].plot(node_times[1:], ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[4].fill_between(node_times[1:], ssp1_ra_r2_d[1:], ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:], alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[4].plot(node_times[1:], ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:] + ssp1_prtp_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[4].fill_between(node_times[1:], ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:], ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:] + ssp1_prtp_r2_d[1:], alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[4].plot(node_times[1:], ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:] + ssp1_prtp_r2_d[1:] + ssp1_tech_r2_d[1:], linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[4].fill_between(node_times[1:], ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:] + ssp1_prtp_r2_d[1:], ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:] + ssp1_prtp_r2_d[1:] + ssp1_tech_r2_d[1:], alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

ax[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=5)

ax[0].set_ylabel("r$^{2}$")
ax[0].set_title("SSP5")
ax[1].set_title("SSP4")
ax[2].set_title("SSP3")
ax[3].set_title("SSP2")
ax[4].set_title("SSP1")

for i in range(5):
    ax[i].set_xlim((node_times[1], node_times[-1]))
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_xlabel("Year")

#fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(20,20), sharey=True)

ax[0].set_ylim((0,1))

alpha = 0.8
lw = 3
l1cind = 0
l2cind = 1
l3cind = 2
l4cind = 3
l5cind = 4

################################
# SSP5
################################
tot_ssp5 = ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:] + ssp5_prtp_r2_d[1:] + ssp5_tech_r2_d[1:]
ax[0].plot(node_times[1:], ssp5_ra_r2_d[1:] * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[0].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_d[1:] * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[0].plot(node_times[1:], (ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[0].fill_between(node_times[1:], ssp5_ra_r2_co[1:]  * tot_ssp5**(-1), 
                   (ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[0].plot(node_times[1:], (ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:] + ssp5_prtp_r2_d[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[0].fill_between(node_times[1:], (ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:]) * tot_ssp5**(-1), 
                  (ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:] + ssp5_prtp_r2_d[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[0].plot(node_times[1:], (ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:] + ssp5_prtp_r2_d[1:] + ssp5_tech_r2_d[1:]) * tot_ssp5**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax[0].fill_between(node_times[1:], (ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:] + ssp5_prtp_r2_d[1:]) * tot_ssp5**(-1), 
                   (ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:] + ssp5_prtp_r2_d[1:] + ssp5_tech_r2_d[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
tot_ssp4 = ssp4_ra_r2_d[1:] + ssp4_eis_r2_d[1:] + ssp4_prtp_r2_d[1:] + ssp4_tech_r2_d[1:]
ax[1].plot(node_times[1:], ssp4_ra_r2_d[1:] * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[1].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp4_ra_r2_d[1:] * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[1].plot(node_times[1:], (ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[1].fill_between(node_times[1:], ssp4_ra_r2_d[1:] * tot_ssp4**(-1), 
                   (ssp4_ra_r2_d[1:] + ssp4_eis_r2_d[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[1].plot(node_times[1:], (ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:] + ssp4_prtp_r2_d[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[1].fill_between(node_times[1:], (ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:]) * tot_ssp4**(-1), 
                  (ssp4_ra_r2_d[1:] + ssp4_eis_r2_d[1:] + ssp4_prtp_r2_d[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[1].plot(node_times[1:], (ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:] + ssp4_prtp_r2_d[1:] + ssp4_tech_r2_d[1:]) * tot_ssp4**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax[1].fill_between(node_times[1:], (ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:] + ssp4_prtp_r2_d[1:]) * tot_ssp4**(-1), 
                   (ssp4_ra_r2_d[1:] + ssp4_eis_r2_d[1:] + ssp4_prtp_r2_d[1:] + ssp4_tech_r2_d[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ssp3_tot = ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:] + ssp3_prtp_r2_d[1:] + ssp3_tech_r2_d[1:]
ax[2].plot(node_times[1:], ssp3_ra_r2_d[1:] * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[2].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp3_ra_r2_d[1:] * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[2].plot(node_times[1:], (ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[2].fill_between(node_times[1:], ssp3_ra_r2_d[1:] * ssp3_tot**(-1), 
                   (ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[2].plot(node_times[1:], (ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:] + ssp3_prtp_r2_d[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[2].fill_between(node_times[1:], (ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:] + ssp3_prtp_r2_d[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[2].plot(node_times[1:], (ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:] + ssp3_prtp_r2_d[1:] + ssp3_tech_r2_d[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[2].fill_between(node_times[1:], (ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:] + ssp3_prtp_r2_d[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:] + ssp3_prtp_r2_d[1:] + ssp3_tech_r2_d[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ssp2_tot = ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:] + ssp2_prtp_r2_d[1:] + ssp2_tech_r2_d[1:]
ax[3].plot(node_times[1:], ssp2_ra_r2_d[1:] * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[3].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp2_ra_r2_d[1:]* ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[3].plot(node_times[1:], (ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:])* ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[3].fill_between(node_times[1:], ssp2_ra_r2_d[1:] * ssp2_tot**(-1), 
                   (ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[3].plot(node_times[1:], (ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:] + ssp2_prtp_r2_d[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[3].fill_between(node_times[1:], (ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:] + ssp2_prtp_r2_d[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[3].plot(node_times[1:], (ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:] + ssp2_prtp_r2_d[1:] + ssp2_tech_r2_d[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[3].fill_between(node_times[1:], (ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:] + ssp2_prtp_r2_d[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:] + ssp2_prtp_r2_d[1:] + ssp2_tech_r2_d[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ssp1_tot = ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:] + ssp1_prtp_r2_d[1:] + ssp1_tech_r2_d[1:]
ax[4].plot(node_times[1:], ssp1_ra_r2_d[1:] * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[4].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_d[1:] * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[4].plot(node_times[1:], (ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[4].fill_between(node_times[1:], ssp1_ra_r2_d[1:] * ssp1_tot**(-1), 
                   (ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l2cind], label="EIS")

ax[4].plot(node_times[1:], (ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:] + ssp1_prtp_r2_d[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[4].fill_between(node_times[1:], (ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:] + ssp1_prtp_r2_d[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l3cind], label="PRTP")

ax[4].plot(node_times[1:], (ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:] + ssp1_prtp_r2_d[1:] + ssp1_tech_r2_d[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[4].fill_between(node_times[1:], (ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:] + ssp1_prtp_r2_d[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:] + ssp1_prtp_r2_d[1:] + ssp1_tech_r2_d[1:]) * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

ax[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=5, fontsize=20)

ax[0].set_ylabel("$r^{2} / r_{tot}^{2}$")
ax[0].set_title("SSP5")
ax[1].set_title("SSP4")
ax[2].set_title("SSP3")
ax[3].set_title("SSP2")
ax[4].set_title("SSP1")

panel_label = ["A", "B", "C", "D", "E"]
for i in range(5):
    ax[i].set_xlim((node_times[1], node_times[-1]))
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_xlabel("Year")
    ax[i].text(2168, 0.88, panel_label[i], fontsize=22, color='white', fontweight='bold')
    ax[i].set_xticks([2030,2100,2200])
    ax[i].set_xticklabels(['2030', '2100', '2200'])
    ax[i].tick_params(axis='both', labelsize=14)

sns.despine(trim=True, offset=10)

if save_fig:
    fig.savefig(''.join([basefile, 'rsquared-dam.png']), dpi=400)

In [ ]:
import matplotlib.transforms as mtransforms

cost_row = ['A', 'B', 'C', 'D', 'E']
t_row = ['F', 'G', 'H', 'I', 'J']
conc_row = ['K', 'L', 'M', 'N', 'O']
dam_row = ['P', 'Q', 'R', 'S', 'T']

fig, ax = plt.subplot_mosaic([cost_row, t_row, conc_row, dam_row], sharex=False,
                             gridspec_kw={'height_ratios': [1, 1, 1, 1], 'width_ratios': [1, 1, 1, 1, 1]},
                            figsize=(25,19))

alpha = 0.8
lw = 3
l1cind = 0
l2cind = 1
l3cind = 2
l4cind = 3
l5cind = 4

################################
# SSP5
################################
tot_ssp5 = ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2 + ssp5_tech_r2
ax['A'].plot(node_times, ssp5_ra_r2 * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['A'].fill_between(node_times, np.zeros_like(node_times), ssp5_ra_r2 * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['A'].plot(node_times, (ssp5_eis_r2 + ssp5_ra_r2)  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['A'].fill_between(node_times, ssp5_ra_r2  * tot_ssp5**(-1), 
                   (ssp5_ra_r2 + ssp5_eis_r2) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['A'].plot(node_times, (ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2)  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['A'].fill_between(node_times, (ssp5_eis_r2 + ssp5_ra_r2) * tot_ssp5**(-1), 
                  (ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['A'].plot(node_times, (ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2 + ssp5_tech_r2) * tot_ssp5**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax['A'].fill_between(node_times, (ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2) * tot_ssp5**(-1), 
                   (ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2 + ssp5_tech_r2) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
tot_ssp4 = ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2 + ssp4_tech_r2
ax['B'].plot(node_times, ssp4_ra_r2 * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['B'].fill_between(node_times, np.zeros_like(node_times), ssp4_ra_r2 * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['B'].plot(node_times, (ssp4_eis_r2 + ssp4_ra_r2) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['B'].fill_between(node_times, ssp4_ra_r2 * tot_ssp4**(-1), 
                   (ssp4_ra_r2 + ssp4_eis_r2) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['B'].plot(node_times, (ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['B'].fill_between(node_times, (ssp4_eis_r2 + ssp4_ra_r2) * tot_ssp4**(-1), 
                  (ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['B'].plot(node_times, (ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2 + ssp4_tech_r2) * tot_ssp4**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax['B'].fill_between(node_times, (ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2) * tot_ssp4**(-1), 
                   (ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2 + ssp4_tech_r2) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ssp3_tot = ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2 + ssp3_tech_r2
ax['C'].plot(node_times, ssp3_ra_r2 * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['C'].fill_between(node_times, np.zeros_like(node_times), ssp3_ra_r2 * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['C'].plot(node_times, (ssp3_eis_r2 + ssp3_ra_r2) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['C'].fill_between(node_times, ssp3_ra_r2 * ssp3_tot**(-1), 
                   (ssp3_ra_r2 + ssp3_eis_r2) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['C'].plot(node_times, (ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['C'].fill_between(node_times, (ssp3_eis_r2 + ssp3_ra_r2) * ssp3_tot**(-1), 
                   (ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['C'].plot(node_times, (ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2 + ssp3_tech_r2) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['C'].fill_between(node_times, (ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2) * ssp3_tot**(-1), 
                   (ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2 + ssp3_tech_r2) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ssp2_tot = ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2 + ssp2_tech_r2
ax['D'].plot(node_times, ssp2_ra_r2 * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['D'].fill_between(node_times, np.zeros_like(node_times), ssp2_ra_r2* ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['D'].plot(node_times, (ssp2_eis_r2 + ssp2_ra_r2)* ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['D'].fill_between(node_times, ssp2_ra_r2 * ssp2_tot**(-1), 
                   (ssp2_ra_r2 + ssp2_eis_r2) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['D'].plot(node_times, (ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['D'].fill_between(node_times, (ssp2_eis_r2 + ssp2_ra_r2) * ssp2_tot**(-1), 
                   (ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['D'].plot(node_times, (ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2 + ssp2_tech_r2) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['D'].fill_between(node_times, (ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2) * ssp2_tot**(-1), 
                   (ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2 + ssp2_tech_r2) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ssp1_tot = ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2 + ssp1_tech_r2
ax['E'].plot(node_times, ssp1_ra_r2 * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['E'].fill_between(node_times, np.zeros_like(node_times), ssp5_ra_r2 * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['E'].plot(node_times, (ssp1_eis_r2 + ssp1_ra_r2) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['E'].fill_between(node_times, ssp1_ra_r2 * ssp1_tot**(-1), 
                   (ssp1_ra_r2 + ssp1_eis_r2) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l2cind], label="EIS")

ax['E'].plot(node_times, (ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['E'].fill_between(node_times, (ssp1_eis_r2 + ssp1_ra_r2) * ssp1_tot**(-1), 
                   (ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l3cind], label="PRTP")

ax['E'].plot(node_times, (ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2 + ssp1_tech_r2) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['E'].fill_between(node_times, (ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2) * ssp1_tot**(-1), 
                   (ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2 + ssp1_tech_r2) * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP5
################################
tot_ssp5 = ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:] + ssp5_prtp_r2_t[1:] + ssp5_tech_r2_t[1:]
ax['F'].plot(node_times[1:], ssp5_ra_r2_t[1:] * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['F'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_t[1:] * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['F'].plot(node_times[1:], (ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['F'].fill_between(node_times[1:], ssp5_ra_r2_t[1:]  * tot_ssp5**(-1), 
                   (ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['F'].plot(node_times[1:], (ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:] + ssp5_prtp_r2_t[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['F'].fill_between(node_times[1:], (ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:]) * tot_ssp5**(-1), 
                  (ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:] + ssp5_prtp_r2_t[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['F'].plot(node_times[1:], (ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:] + ssp5_prtp_r2_t[1:] + ssp5_tech_r2_t[1:]) * tot_ssp5**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax['F'].fill_between(node_times[1:], (ssp5_eis_r2_t[1:] + ssp5_ra_r2_t[1:] + ssp5_prtp_r2_t[1:]) * tot_ssp5**(-1), 
                   (ssp5_ra_r2_t[1:] + ssp5_eis_r2_t[1:] + ssp5_prtp_r2_t[1:] + ssp5_tech_r2_t[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
tot_ssp4 = ssp4_ra_r2_t[1:] + ssp4_eis_r2_t[1:] + ssp4_prtp_r2_t[1:] + ssp4_tech_r2_t[1:]
ax['G'].plot(node_times[1:], ssp4_ra_r2_t[1:] * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['G'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp4_ra_r2_t[1:] * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['G'].plot(node_times[1:], (ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['G'].fill_between(node_times[1:], ssp4_ra_r2_t[1:] * tot_ssp4**(-1), 
                   (ssp4_ra_r2_t[1:] + ssp4_eis_r2_t[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['G'].plot(node_times[1:], (ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:] + ssp4_prtp_r2_t[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['G'].fill_between(node_times[1:], (ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:]) * tot_ssp4**(-1), 
                  (ssp4_ra_r2_t[1:] + ssp4_eis_r2_t[1:] + ssp4_prtp_r2_t[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['G'].plot(node_times[1:], (ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:] + ssp4_prtp_r2_t[1:] + ssp4_tech_r2_t[1:]) * tot_ssp4**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax['G'].fill_between(node_times[1:], (ssp4_eis_r2_t[1:] + ssp4_ra_r2_t[1:] + ssp4_prtp_r2_t[1:]) * tot_ssp4**(-1), 
                   (ssp4_ra_r2_t[1:] + ssp4_eis_r2_t[1:] + ssp4_prtp_r2_t[1:] + ssp4_tech_r2_t[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ssp3_tot = ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:] + ssp3_prtp_r2_t[1:] + ssp3_tech_r2_t[1:]
ax['H'].plot(node_times[1:], ssp3_ra_r2_t[1:] * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['H'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp3_ra_r2_t[1:] * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['H'].plot(node_times[1:], (ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['H'].fill_between(node_times[1:], ssp3_ra_r2_t[1:] * ssp3_tot**(-1), 
                   (ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['H'].plot(node_times[1:], (ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:] + ssp3_prtp_r2_t[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['H'].fill_between(node_times[1:], (ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:] + ssp3_prtp_r2_t[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['H'].plot(node_times[1:], (ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:] + ssp3_prtp_r2_t[1:] + ssp3_tech_r2_t[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['H'].fill_between(node_times[1:], (ssp3_eis_r2_t[1:] + ssp3_ra_r2_t[1:] + ssp3_prtp_r2_t[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_t[1:] + ssp3_eis_r2_t[1:] + ssp3_prtp_r2_t[1:] + ssp3_tech_r2_t[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ssp2_tot = ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:] + ssp2_prtp_r2_t[1:] + ssp2_tech_r2_t[1:]
ax['I'].plot(node_times[1:], ssp2_ra_r2_t[1:] * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['I'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp2_ra_r2_t[1:]* ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['I'].plot(node_times[1:], (ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:])* ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['I'].fill_between(node_times[1:], ssp2_ra_r2_t[1:] * ssp2_tot**(-1), 
                   (ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['I'].plot(node_times[1:], (ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:] + ssp2_prtp_r2_t[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['I'].fill_between(node_times[1:], (ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:] + ssp2_prtp_r2_t[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['I'].plot(node_times[1:], (ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:] + ssp2_prtp_r2_t[1:] + ssp2_tech_r2_t[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['I'].fill_between(node_times[1:], (ssp2_eis_r2_t[1:] + ssp2_ra_r2_t[1:] + ssp2_prtp_r2_t[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_t[1:] + ssp2_eis_r2_t[1:] + ssp2_prtp_r2_t[1:] + ssp2_tech_r2_t[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ssp1_tot = ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:] + ssp1_prtp_r2_t[1:] + ssp1_tech_r2_t[1:]
ax['J'].plot(node_times[1:], ssp1_ra_r2_t[1:] * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['J'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_t[1:] * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['J'].plot(node_times[1:], (ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['J'].fill_between(node_times[1:], ssp1_ra_r2_t[1:] * ssp1_tot**(-1), 
                   (ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l2cind], label="EIS")

ax['J'].plot(node_times[1:], (ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:] + ssp1_prtp_r2_t[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['J'].fill_between(node_times[1:], (ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:] + ssp1_prtp_r2_t[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l3cind], label="PRTP")

ax['J'].plot(node_times[1:], (ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:] + ssp1_prtp_r2_t[1:] + ssp1_tech_r2_t[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['J'].fill_between(node_times[1:], (ssp1_eis_r2_t[1:] + ssp1_ra_r2_t[1:] + ssp1_prtp_r2_t[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_t[1:] + ssp1_eis_r2_t[1:] + ssp1_prtp_r2_t[1:] + ssp1_tech_r2_t[1:]) * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP5
################################
tot_ssp5 = ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:] + ssp5_prtp_r2_co[1:] + ssp5_tech_r2_co[1:]
ax['K'].plot(node_times[1:], ssp5_ra_r2_co[1:] * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['K'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_co[1:] * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['K'].plot(node_times[1:], (ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['K'].fill_between(node_times[1:], ssp5_ra_r2_co[1:]  * tot_ssp5**(-1), 
                   (ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['K'].plot(node_times[1:], (ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:] + ssp5_prtp_r2_co[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['K'].fill_between(node_times[1:], (ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:]) * tot_ssp5**(-1), 
                  (ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:] + ssp5_prtp_r2_co[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['K'].plot(node_times[1:], (ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:] + ssp5_prtp_r2_co[1:] + ssp5_tech_r2_co[1:]) * tot_ssp5**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax['K'].fill_between(node_times[1:], (ssp5_eis_r2_co[1:] + ssp5_ra_r2_co[1:] + ssp5_prtp_r2_co[1:]) * tot_ssp5**(-1), 
                   (ssp5_ra_r2_co[1:] + ssp5_eis_r2_co[1:] + ssp5_prtp_r2_co[1:] + ssp5_tech_r2_co[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
tot_ssp4 = ssp4_ra_r2_co[1:] + ssp4_eis_r2_co[1:] + ssp4_prtp_r2_co[1:] + ssp4_tech_r2_co[1:]
ax['L'].plot(node_times[1:], ssp4_ra_r2_co[1:] * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['L'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp4_ra_r2_co[1:] * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['L'].plot(node_times[1:], (ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['L'].fill_between(node_times[1:], ssp4_ra_r2_co[1:] * tot_ssp4**(-1), 
                   (ssp4_ra_r2_co[1:] + ssp4_eis_r2_co[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['L'].plot(node_times[1:], (ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:] + ssp4_prtp_r2_co[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['L'].fill_between(node_times[1:], (ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:]) * tot_ssp4**(-1), 
                  (ssp4_ra_r2_co[1:] + ssp4_eis_r2_co[1:] + ssp4_prtp_r2_co[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['L'].plot(node_times[1:], (ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:] + ssp4_prtp_r2_co[1:] + ssp4_tech_r2_co[1:]) * tot_ssp4**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax['L'].fill_between(node_times[1:], (ssp4_eis_r2_co[1:] + ssp4_ra_r2_co[1:] + ssp4_prtp_r2_co[1:]) * tot_ssp4**(-1), 
                   (ssp4_ra_r2_co[1:] + ssp4_eis_r2_co[1:] + ssp4_prtp_r2_co[1:] + ssp4_tech_r2_co[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ssp3_tot = ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:] + ssp3_prtp_r2_co[1:] + ssp3_tech_r2_co[1:]
ax['M'].plot(node_times[1:], ssp3_ra_r2_co[1:] * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['M'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp3_ra_r2_co[1:] * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['M'].plot(node_times[1:], (ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['M'].fill_between(node_times[1:], ssp3_ra_r2_co[1:] * ssp3_tot**(-1), 
                   (ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['M'].plot(node_times[1:], (ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:] + ssp3_prtp_r2_co[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['M'].fill_between(node_times[1:], (ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:] + ssp3_prtp_r2_co[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['M'].plot(node_times[1:], (ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:] + ssp3_prtp_r2_co[1:] + ssp3_tech_r2_co[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['M'].fill_between(node_times[1:], (ssp3_eis_r2_co[1:] + ssp3_ra_r2_co[1:] + ssp3_prtp_r2_co[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_co[1:] + ssp3_eis_r2_co[1:] + ssp3_prtp_r2_co[1:] + ssp3_tech_r2_co[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ssp2_tot = ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:] + ssp2_prtp_r2_co[1:] + ssp2_tech_r2_co[1:]
ax['N'].plot(node_times[1:], ssp2_ra_r2_co[1:] * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['N'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp2_ra_r2_co[1:]* ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['N'].plot(node_times[1:], (ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:])* ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['N'].fill_between(node_times[1:], ssp2_ra_r2_co[1:] * ssp2_tot**(-1), 
                   (ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['N'].plot(node_times[1:], (ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:] + ssp2_prtp_r2_co[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['N'].fill_between(node_times[1:], (ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:] + ssp2_prtp_r2_co[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['N'].plot(node_times[1:], (ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:] + ssp2_prtp_r2_co[1:] + ssp2_tech_r2_co[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['N'].fill_between(node_times[1:], (ssp2_eis_r2_co[1:] + ssp2_ra_r2_co[1:] + ssp2_prtp_r2_co[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_co[1:] + ssp2_eis_r2_co[1:] + ssp2_prtp_r2_co[1:] + ssp2_tech_r2_co[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ssp1_tot = ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:] + ssp1_prtp_r2_co[1:] + ssp1_tech_r2_co[1:]
ax['O'].plot(node_times[1:], ssp1_ra_r2_co[1:] * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['O'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_co[1:] * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['O'].plot(node_times[1:], (ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['O'].fill_between(node_times[1:], ssp1_ra_r2_co[1:] * ssp1_tot**(-1), 
                   (ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l2cind], label="EIS")

ax['O'].plot(node_times[1:], (ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:] + ssp1_prtp_r2_co[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['O'].fill_between(node_times[1:], (ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:] + ssp1_prtp_r2_co[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l3cind], label="PRTP")

ax['O'].plot(node_times[1:], (ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:] + ssp1_prtp_r2_co[1:] + ssp1_tech_r2_co[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['O'].fill_between(node_times[1:], (ssp1_eis_r2_co[1:] + ssp1_ra_r2_co[1:] + ssp1_prtp_r2_co[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_co[1:] + ssp1_eis_r2_co[1:] + ssp1_prtp_r2_co[1:] + ssp1_tech_r2_co[1:]) * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP5
################################
tot_ssp5 = ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:] + ssp5_prtp_r2_d[1:] + ssp5_tech_r2_d[1:]
ax['P'].plot(node_times[1:], ssp5_ra_r2_d[1:] * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['P'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_d[1:] * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['P'].plot(node_times[1:], (ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['P'].fill_between(node_times[1:], ssp5_ra_r2_co[1:]  * tot_ssp5**(-1), 
                   (ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['P'].plot(node_times[1:], (ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:] + ssp5_prtp_r2_d[1:])  * tot_ssp5**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['P'].fill_between(node_times[1:], (ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:]) * tot_ssp5**(-1), 
                  (ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:] + ssp5_prtp_r2_d[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['P'].plot(node_times[1:], (ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:] + ssp5_prtp_r2_d[1:] + ssp5_tech_r2_d[1:]) * tot_ssp5**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax['P'].fill_between(node_times[1:], (ssp5_eis_r2_d[1:] + ssp5_ra_r2_d[1:] + ssp5_prtp_r2_d[1:]) * tot_ssp5**(-1), 
                   (ssp5_ra_r2_d[1:] + ssp5_eis_r2_d[1:] + ssp5_prtp_r2_d[1:] + ssp5_tech_r2_d[1:]) * tot_ssp5**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP4
################################
tot_ssp4 = ssp4_ra_r2_d[1:] + ssp4_eis_r2_d[1:] + ssp4_prtp_r2_d[1:] + ssp4_tech_r2_d[1:]
ax['Q'].plot(node_times[1:], ssp4_ra_r2_d[1:] * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['Q'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp4_ra_r2_d[1:] * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['Q'].plot(node_times[1:], (ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['Q'].fill_between(node_times[1:], ssp4_ra_r2_d[1:] * tot_ssp4**(-1), 
                   (ssp4_ra_r2_d[1:] + ssp4_eis_r2_d[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['Q'].plot(node_times[1:], (ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:] + ssp4_prtp_r2_d[1:]) * tot_ssp4**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['Q'].fill_between(node_times[1:], (ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:]) * tot_ssp4**(-1), 
                  (ssp4_ra_r2_d[1:] + ssp4_eis_r2_d[1:] + ssp4_prtp_r2_d[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['Q'].plot(node_times[1:], (ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:] + ssp4_prtp_r2_d[1:] + ssp4_tech_r2_d[1:]) * tot_ssp4**(-1), 
           linestyle='dashed', linewidth=lw, color=color_list[l4cind])
ax['Q'].fill_between(node_times[1:], (ssp4_eis_r2_d[1:] + ssp4_ra_r2_d[1:] + ssp4_prtp_r2_d[1:]) * tot_ssp4**(-1), 
                   (ssp4_ra_r2_d[1:] + ssp4_eis_r2_d[1:] + ssp4_prtp_r2_d[1:] + ssp4_tech_r2_d[1:]) * tot_ssp4**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP3
################################
ssp3_tot = ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:] + ssp3_prtp_r2_d[1:] + ssp3_tech_r2_d[1:]
ax['R'].plot(node_times[1:], ssp3_ra_r2_d[1:] * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['R'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp3_ra_r2_d[1:] * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['R'].plot(node_times[1:], (ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['R'].fill_between(node_times[1:], ssp3_ra_r2_d[1:] * ssp3_tot**(-1), 
                   (ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['R'].plot(node_times[1:], (ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:] + ssp3_prtp_r2_d[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['R'].fill_between(node_times[1:], (ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:] + ssp3_prtp_r2_d[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['R'].plot(node_times[1:], (ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:] + ssp3_prtp_r2_d[1:] + ssp3_tech_r2_d[1:]) * ssp3_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['R'].fill_between(node_times[1:], (ssp3_eis_r2_d[1:] + ssp3_ra_r2_d[1:] + ssp3_prtp_r2_d[1:]) * ssp3_tot**(-1), 
                   (ssp3_ra_r2_d[1:] + ssp3_eis_r2_d[1:] + ssp3_prtp_r2_d[1:] + ssp3_tech_r2_d[1:]) * ssp3_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")

################################
# SSP2
################################
ssp2_tot = ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:] + ssp2_prtp_r2_d[1:] + ssp2_tech_r2_d[1:]
ax['S'].plot(node_times[1:], ssp2_ra_r2_d[1:] * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['S'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp2_ra_r2_d[1:]* ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax['S'].plot(node_times[1:], (ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:])* ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['S'].fill_between(node_times[1:], ssp2_ra_r2_d[1:] * ssp2_tot**(-1), 
                   (ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax['S'].plot(node_times[1:], (ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:] + ssp2_prtp_r2_d[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['S'].fill_between(node_times[1:], (ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:] + ssp2_prtp_r2_d[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax['S'].plot(node_times[1:], (ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:] + ssp2_prtp_r2_d[1:] + ssp2_tech_r2_d[1:]) * ssp2_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['S'].fill_between(node_times[1:], (ssp2_eis_r2_d[1:] + ssp2_ra_r2_d[1:] + ssp2_prtp_r2_d[1:]) * ssp2_tot**(-1), 
                   (ssp2_ra_r2_d[1:] + ssp2_eis_r2_d[1:] + ssp2_prtp_r2_d[1:] + ssp2_tech_r2_d[1:]) * ssp2_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Tech. Growth")


################################
# SSP1
################################
ssp1_tot = ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:] + ssp1_prtp_r2_d[1:] + ssp1_tech_r2_d[1:]
ax['T'].plot(node_times[1:], ssp1_ra_r2_d[1:] * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax['T'].fill_between(node_times[1:], np.zeros_like(node_times[1:]), ssp5_ra_r2_d[1:] * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l1cind], label="Epstein-Zin Risk Aversion, $\psi$")

ax['T'].plot(node_times[1:], (ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax['T'].fill_between(node_times[1:], ssp1_ra_r2_d[1:] * ssp1_tot**(-1), 
                   (ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l2cind], label="Elasticity of Intertemporal Substitution, $\sigma$")

ax['T'].plot(node_times[1:], (ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:] + ssp1_prtp_r2_d[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax['T'].fill_between(node_times[1:], (ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:] + ssp1_prtp_r2_d[1:]) * ssp1_tot**(-1), alpha=alpha, 
                    color=color_list[l3cind], label="Pure rate of time preference, $\rho$")

ax['T'].plot(node_times[1:], (ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:] + ssp1_prtp_r2_d[1:] + ssp1_tech_r2_d[1:]) * ssp1_tot**(-1), linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax['T'].fill_between(node_times[1:], (ssp1_eis_r2_d[1:] + ssp1_ra_r2_d[1:] + ssp1_prtp_r2_d[1:]) * ssp1_tot**(-1), 
                   (ssp1_ra_r2_d[1:] + ssp1_eis_r2_d[1:] + ssp1_prtp_r2_d[1:] + ssp1_tech_r2_d[1:]) * ssp1_tot**(-1), alpha=alpha, 
                color=color_list[l4cind], label="Technology Growth, $\varphi = \varphi_{0} + \varphi_{1} X_{t}")

ax['R'].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), fancybox=True, shadow=True, ncol=5, fontsize=20)


for i in cost_row:
    ax[i].set_xlim((node_times[0], node_times[-1]))
    ax[i].set_xticks([2020, 2100, 2200])
    ax[i].set_xticklabels(['2020', '', ''])
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_ylim((0,1))
    
for i in t_row:
    ax[i].set_xlim((node_times[1], node_times[-1]))
    ax[i].set_xticks([2030, 2100, 2200])
    ax[i].set_xticklabels([])
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_ylim((0,1))
    
for i in conc_row:
    ax[i].set_xlim((node_times[1], node_times[-1]))
    ax[i].set_xticks([2030, 2100, 2200])
    ax[i].set_xticklabels([])
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_ylim((0,1))
    
for i in dam_row:
    ax[i].set_xlim((node_times[1], node_times[-1]))
    ax[i].set_xticks([2030, 2100, 2200])
    ax[i].set_xticklabels(['2030', '2100', '2200'])
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    
for i in cost_row[1:] + t_row[1:] + conc_row[1:] + dam_row[1:]:
    ax[i].set_yticklabels([])

sns.despine(trim=True, offset=10)

ax['A'].set_ylabel("$r^{2} / r_{tot}^{2}$")
ax['F'].set_ylabel("$r^{2} / r_{tot}^{2}$")
ax['K'].set_ylabel("$r^{2} / r_{tot}^{2}$")
ax['P'].set_ylabel("$r^{2} / r_{tot}^{2}$")


ax['A'].set_title("Cost              SSP5$-$8.5")
ax['B'].set_title("SSP4$-6.0$")
ax['C'].set_title("SSP3$-$7.0")
ax['D'].set_title("SSP2$-$4.5")
ax['E'].set_title("SSP1$-$1.9")
ax['F'].set_title("Temperature")
ax['K'].set_title("CO$_2$ concentrations")
ax['P'].set_title("Damages")

# label panels
for label, ax in ax.items():
    # label physical distance in and down:
    trans = mtransforms.ScaledTranslation(2.9, -0.13, fig.dpi_scale_trans)
    ax.text(0.0, 1.0, label, color='white', transform=ax.transAxes + trans, fontsize=22, fontweight='bold',
            verticalalignment='top', bbox=dict(facecolor='none', edgecolor='none', pad=1))
    ax.set_ylim((0,1))

fig.subplots_adjust(wspace=0.2, hspace=0.3)

if save_fig:
    fig.savefig(''.join([basefile, 'rsquared-all.png']), dpi=400)

# Old stuff

Where we treated endogeneous and exogeneous as their own parameters, and didn't take into account agent behavior in the calculation of the technological growth rate.

In [ ]:
fig, ax = plt.subplots(5, 6, figsize=(30, 20))

ax[0,0].set_title("2020")
ax[0,1].set_title("2030")
ax[0,2].set_title("2060")
ax[0,3].set_title("2100")
ax[0,4].set_title("2150")
ax[0,5].set_title("2200")

###########################
## RISK AVERSION ROW
###########################
make_panel_w_regression(ax, 0, 0, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2020, ssp4_c2020, ssp3_c2020, ssp2_c2020, ssp1_c2020, 
           "Risk Aversion", "Cost\n(2020 $USD)")

make_panel_w_regression(ax, 0, 1, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2030, ssp4_c2030, ssp3_c2030, ssp2_c2030, ssp1_c2030, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 2, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2060, ssp4_c2060, ssp3_c2060, ssp2_c2060, ssp1_c2060, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 3, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2100, ssp4_c2100, ssp3_c2100, ssp2_c2100, ssp1_c2100, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 4, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2150, ssp4_c2150, ssp3_c2150, ssp2_c2150, ssp1_c2150, 
           "Risk Aversion", None)

make_panel_w_regression(ax, 0, 5, ssp5_ra, ssp4_ra, ssp3_ra, ssp2_ra, ssp1_ra, ssp5_c2200, ssp4_c2200, ssp3_c2200, ssp2_c2200, ssp1_c2200, 
           "Risk Aversion", None)

###########################
# EIS ROW
###########################
make_panel_w_regression(ax, 1, 0, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2020, ssp4_c2020, ssp3_c2020, ssp2_c2020, ssp1_c2020, 
           "EIS", "Cost\n(2020 $USD)")

make_panel_w_regression(ax, 1, 1, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2030, ssp4_c2030, ssp3_c2030, ssp2_c2030, ssp1_c2030, 
           "EIS", None)

make_panel_w_regression(ax, 1, 2, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2060, ssp4_c2060, ssp3_c2060, ssp2_c2060, ssp1_c2060, 
           "EIS", None)

make_panel_w_regression(ax, 1, 3, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2100, ssp4_c2100, ssp3_c2100, ssp2_c2100, ssp1_c2100, 
           "EIS", None)

make_panel_w_regression(ax, 1, 4, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2150, ssp4_c2150, ssp3_c2150, ssp2_c2150, ssp1_c2150, 
           "EIS", None)

make_panel_w_regression(ax, 1, 5, ssp5_eis, ssp4_eis, ssp3_eis, ssp2_eis, ssp1_eis, ssp5_c2200, ssp4_c2200, ssp3_c2200, ssp2_c2200, ssp1_c2200, 
           "EIS", None)

###########################
# PRTP ROW
###########################
make_panel_w_regression(ax, 2, 0, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2020, ssp4_c2020, ssp3_c2020, ssp2_c2020, ssp1_c2020, 
           r"$\delta$ (%)", "Cost\n(2020 $USD)")

make_panel_w_regression(ax, 2, 1, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2030, ssp4_c2030, ssp3_c2030, ssp2_c2030, ssp1_c2030, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 2, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2060, ssp4_c2060, ssp3_c2060, ssp2_c2060, ssp1_c2060, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 3, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2100, ssp4_c2100, ssp3_c2100, ssp2_c2100, ssp1_c2100, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 4, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2150, ssp4_c2150, ssp3_c2150, ssp2_c2150, ssp1_c2150, 
           r"$\delta$ (%)", None)

make_panel_w_regression(ax, 2, 5, ssp5_prtp * 100, ssp4_prtp * 100, ssp3_prtp * 100, ssp2_prtp * 100, ssp1_prtp * 100, ssp5_c2200, ssp4_c2200, ssp3_c2200, ssp2_c2200, ssp1_c2200, 
           r"$\delta$ (%)", None)

###########################
# EX TECH GROWTH ROW
###########################
make_panel_w_regression(ax, 3, 0, ssp5_ex, ssp4_ex, ssp3_ex, ssp2_ex, ssp1_ex, ssp5_c2020, ssp4_c2020, ssp3_c2020, ssp2_c2020, ssp1_c2020, 
           "Ex. Tech Growth (%)", "Cost\n(2020 $USD)")

make_panel_w_regression(ax, 3, 1, ssp5_ex, ssp4_ex, ssp3_ex, ssp2_ex, ssp1_ex, ssp5_c2030, ssp4_c2030, ssp3_c2030, ssp2_c2030, ssp1_c2030, 
           "Ex. Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 2, ssp5_ex, ssp4_ex, ssp3_ex, ssp2_ex, ssp1_ex, ssp5_c2060, ssp4_c2060, ssp3_c2060, ssp2_c2060, ssp1_c2060, 
           "Ex. Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 3, ssp5_ex, ssp4_ex, ssp3_ex, ssp2_ex, ssp1_ex, ssp5_c2100, ssp4_c2100, ssp3_c2100, ssp2_c2100, ssp1_c2100, 
           "Ex. Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 4, ssp5_ex, ssp4_ex, ssp3_ex, ssp2_ex, ssp1_ex, ssp5_c2150, ssp4_c2150, ssp3_c2150, ssp2_c2150, ssp1_c2150, 
           "Ex. Tech Growth (%)", None)

make_panel_w_regression(ax, 3, 5, ssp5_ex, ssp4_ex, ssp3_ex, ssp2_ex, ssp1_ex, ssp5_c2200, ssp4_c2200, ssp3_c2200, ssp2_c2200, ssp1_c2200, 
           "Ex. Tech Growth (%)", None)

###########################
# END TECH GROWTH ROW
###########################
make_panel_w_regression(ax, 4, 0, ssp5_end, ssp4_end, ssp3_end, ssp2_end, ssp1_end, ssp5_c2020, ssp4_c2020, ssp3_c2020, ssp2_c2020, ssp1_c2020, 
           "End. Tech Growth (%)", "Cost\n(2020 $USD)")

make_panel_w_regression(ax, 4, 1, ssp5_end, ssp4_end, ssp3_end, ssp2_end, ssp1_end, ssp5_c2030, ssp4_c2030, ssp3_c2030, ssp2_c2030, ssp1_c2030, 
           "End. Tech Growth (%)", None)

make_panel_w_regression(ax, 4, 2, ssp5_end, ssp4_end, ssp3_end, ssp2_end, ssp1_end, ssp5_c2060, ssp4_c2060, ssp3_c2060, ssp2_c2060, ssp1_c2060, 
           "End. Tech Growth (%)", None)

make_panel_w_regression(ax, 4, 3, ssp5_end, ssp4_end, ssp3_end, ssp2_end, ssp1_end, ssp5_c2100, ssp4_c2100, ssp3_c2100, ssp2_c2100, ssp1_c2100, 
           "End. Tech Growth (%)", None)

make_panel_w_regression(ax, 4, 4, ssp5_end, ssp4_end, ssp3_end, ssp2_end, ssp1_end, ssp5_c2150, ssp4_c2150, ssp3_c2150, ssp2_c2150, ssp1_c2150, 
           "End. Tech Growth (%)", None)

make_panel_w_regression(ax, 4, 5, ssp5_end, ssp4_end, ssp3_end, ssp2_end, ssp1_end, ssp5_c2200, ssp4_c2200, ssp3_c2200, ssp2_c2200, ssp1_c2200, 
           "End. Tech Growth (%)", None)

fig.tight_layout()

In [ ]:
ssp5_ex_r2 = get_r2_in_time(ssp5_ex, ssp5_paths)
ssp5_end_r2 = get_r2_in_time(ssp5_end, ssp5_paths)

ssp4_ex_r2 = get_r2_in_time(ssp4_ex, ssp4_paths)
ssp4_end_r2 = get_r2_in_time(ssp4_end, ssp4_paths)

ssp3_ex_r2 = get_r2_in_time(ssp3_ex, ssp3_paths)
ssp3_end_r2 = get_r2_in_time(ssp3_end, ssp3_paths)

ssp2_ex_r2 = get_r2_in_time(ssp2_ex, ssp2_paths)
ssp2_end_r2 = get_r2_in_time(ssp2_end, ssp2_paths)

ssp1_ex_r2 = get_r2_in_time(ssp1_ex, ssp1_paths)
ssp1_end_r2 = get_r2_in_time(ssp1_end, ssp1_paths)

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(20,20), sharey=True)

ax[0].set_ylim((0,1))

alpha = 0.8
lw = 3
l1cind = 0
l2cind = 1
l3cind = 2
l4cind = 3
l5cind = 4

################################
# SSP5
################################
ax[0].plot(node_times, ssp5_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[0].fill_between(node_times, np.zeros_like(node_times), ssp5_ra_r2, alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[0].plot(node_times, ssp5_eis_r2 + ssp5_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[0].fill_between(node_times, ssp5_ra_r2, ssp5_ra_r2 + ssp5_eis_r2, alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[0].plot(node_times, ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[0].fill_between(node_times, ssp5_eis_r2 + ssp5_ra_r2, ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2, alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[0].plot(node_times, ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2 + ssp5_ex_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[0].fill_between(node_times, ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2, ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2 + ssp5_ex_r2, alpha=alpha, 
                color=color_list[l4cind], label="Ex. Tech. Growth")

ax[0].plot(node_times, ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2 + ssp5_ex_r2 + ssp5_end_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l5cind])
ax[0].fill_between(node_times, ssp5_eis_r2 + ssp5_ra_r2 + ssp5_prtp_r2 + ssp5_ex_r2, ssp5_ra_r2 + ssp5_eis_r2 + ssp5_prtp_r2 + ssp5_ex_r2 + ssp5_end_r2, alpha=alpha, 
                color=color_list[l5cind], label="End. Tech. Growth")

################################
# SSP4
################################
ax[1].plot(node_times, ssp4_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[1].fill_between(node_times, np.zeros_like(node_times), ssp4_ra_r2, alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[1].plot(node_times, ssp4_eis_r2 + ssp4_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[1].fill_between(node_times, ssp4_ra_r2, ssp5_ra_r2 + ssp4_eis_r2, alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[1].plot(node_times, ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[1].fill_between(node_times, ssp4_eis_r2 + ssp4_ra_r2, ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2, alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[1].plot(node_times, ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2 + ssp4_ex_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[1].fill_between(node_times, ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2, ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2 + ssp4_ex_r2, alpha=alpha, 
                color=color_list[l4cind], label="Ex. Tech. Growth")

ax[1].plot(node_times, ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2 + ssp4_ex_r2 + ssp4_end_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l5cind])
ax[1].fill_between(node_times, ssp4_eis_r2 + ssp4_ra_r2 + ssp4_prtp_r2 + ssp4_ex_r2, ssp4_ra_r2 + ssp4_eis_r2 + ssp4_prtp_r2 + ssp4_ex_r2 + ssp4_end_r2, alpha=alpha, 
                color=color_list[l5cind], label="End. Tech. Growth")

################################
# SSP3
################################
ax[2].plot(node_times, ssp3_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[2].fill_between(node_times, np.zeros_like(node_times), ssp3_ra_r2, alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[2].plot(node_times, ssp3_eis_r2 + ssp3_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[2].fill_between(node_times, ssp3_ra_r2, ssp3_ra_r2 + ssp3_eis_r2, alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[2].plot(node_times, ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[2].fill_between(node_times, ssp3_eis_r2 + ssp3_ra_r2, ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2, alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[2].plot(node_times, ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2 + ssp3_ex_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[2].fill_between(node_times, ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2, ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2 + ssp3_ex_r2, alpha=alpha, 
                color=color_list[l4cind], label="Ex. Tech. Growth")

ax[2].plot(node_times, ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2 + ssp3_ex_r2 + ssp3_end_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l5cind])
ax[2].fill_between(node_times, ssp3_eis_r2 + ssp3_ra_r2 + ssp3_prtp_r2 + ssp3_ex_r2, ssp3_ra_r2 + ssp3_eis_r2 + ssp3_prtp_r2 + ssp3_ex_r2 + ssp3_end_r2, alpha=alpha, 
                color=color_list[l5cind], label="End. Tech. Growth")

################################
# SSP2
################################
ax[3].plot(node_times, ssp2_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[3].fill_between(node_times, np.zeros_like(node_times), ssp2_ra_r2, alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[3].plot(node_times, ssp2_eis_r2 + ssp2_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[3].fill_between(node_times, ssp2_ra_r2, ssp2_ra_r2 + ssp2_eis_r2, alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[3].plot(node_times, ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[3].fill_between(node_times, ssp2_eis_r2 + ssp2_ra_r2, ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2, alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[3].plot(node_times, ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2 + ssp2_ex_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[3].fill_between(node_times, ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2, ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2 + ssp2_ex_r2, alpha=alpha, 
                color=color_list[l4cind], label="Ex. Tech. Growth")

ax[3].plot(node_times, ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2 + ssp2_ex_r2 + ssp2_end_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l5cind])
ax[3].fill_between(node_times, ssp2_eis_r2 + ssp2_ra_r2 + ssp2_prtp_r2 + ssp2_ex_r2, ssp2_ra_r2 + ssp2_eis_r2 + ssp2_prtp_r2 + ssp2_ex_r2 + ssp2_end_r2, alpha=alpha, 
                color=color_list[l5cind], label="End. Tech. Growth")

################################
# SSP1
################################
ax[4].plot(node_times, ssp1_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l1cind])
ax[4].fill_between(node_times, np.zeros_like(node_times), ssp5_ra_r2, alpha=alpha, 
                color=color_list[l1cind], label="RA")

ax[4].plot(node_times, ssp1_eis_r2 + ssp1_ra_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l2cind])
ax[4].fill_between(node_times, ssp1_ra_r2, ssp1_ra_r2 + ssp1_eis_r2, alpha=alpha, 
                color=color_list[l2cind], label="EIS")

ax[4].plot(node_times, ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l3cind])
ax[4].fill_between(node_times, ssp1_eis_r2 + ssp1_ra_r2, ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2, alpha=alpha, 
                color=color_list[l3cind], label="PRTP")

ax[4].plot(node_times, ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2 + ssp1_ex_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l4cind])
ax[4].fill_between(node_times, ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2, ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2 + ssp1_ex_r2, alpha=alpha, 
                color=color_list[l4cind], label="Ex. Tech. Growth")

ax[4].plot(node_times, ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2 + ssp1_ex_r2 + ssp1_end_r2, linestyle='dashed', linewidth=lw, 
        color=color_list[l5cind])
ax[4].fill_between(node_times, ssp1_eis_r2 + ssp1_ra_r2 + ssp1_prtp_r2 + ssp1_ex_r2, ssp1_ra_r2 + ssp1_eis_r2 + ssp1_prtp_r2 + ssp1_ex_r2 + ssp1_end_r2, alpha=alpha, 
                color=color_list[l5cind], label="End. Tech. Growth")

ax[2].legend(loc='upper center', bbox_to_anchor=(0.5, -0.25), fancybox=True, shadow=True, ncol=5)

ax[0].set_ylabel("r$^{2}$")
ax[0].set_title("SSP5")
ax[1].set_title("SSP4")
ax[2].set_title("SSP3")
ax[3].set_title("SSP2")
ax[4].set_title("SSP1")

for i in range(5):
    ax[i].set_xlim((node_times[0], node_times[-1]))
    ax[i].set_aspect(1.0/ax[i].get_data_ratio(), adjustable='box')
    ax[i].set_xlabel("Year")

#fig.tight_layout()